In [ ]:
# import libraries
import os
import tweepy as tw
import numpy as np
import pandas as pd
import re
import string
import nltk

nltk.download('words')
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


**Collect Tweets Using Twitter API**

In [ ]:
# api keys
consumer_key= 'n8j0jOJCkAITB8umNDSaklipb'
consumer_secret= 'kuJn5KbE1mNaMTLZNYxJKviNWBSzDGZiVwA9SOq6p0tKRWpDXr'
access_token= '1451064720484540421-bnuLCqzhUsswezrvmJoTPpXDCwQnKl'
access_token_secret= 'BPpWZgXVr6eOgmXI9HfaODEUX0tXDx4CQVSk9q2j7we6K'

In [ ]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [ ]:
# search by keyword
# Define the search term and the date_since date as variables
search_words = "vaksin"
date_since = "2020-1-25"

In [ ]:
# search_query = "#covid19malaysia -filter:retweets"

In [ ]:
# Collect tweets
tweets_api = tw.Cursor(api.search,
              q=search_words,
              lang="id",
              since=date_since,
              location="Malaysia").items(10000)

# Iterate and print tweets
tweets = []
data = []

for tweet in tweets_api:
    tweets.append(tweet)
    data.extend([[tweet.user.id, tweet.user.screen_name, tweet.created_at, tweet.text]])

print(data)

[[359329138, 'dindamedina', datetime.datetime(2022, 4, 21, 13, 53, 5), 'Tadi pagi baru vaksin yang ketiga dan nggak ada efek apa-apa sampai sore. Sekarang baru kerasa nyeri-nyeri di badan.'], [446174007, 'Indra_zill', datetime.datetime(2022, 4, 21, 13, 53, 5), '@popienastiti1 Gaji udah dalam bentuk makanan bergizi, akomodasi, dan vaksin.'], [1451457302628765703, 'nb1957_', datetime.datetime(2022, 4, 21, 13, 53, 5), 'RT @deehandhanin: ruu tpks disahkan. vaksin kanker serviks diwajibkan &amp; digratiskan. uji materi permendikbud ppks ditolak.\n\nselamat hari ka…'], [1021087094, 'NugroDwi', datetime.datetime(2022, 4, 21, 13, 52, 58), 'RT @GratisTerbaik: "Jika masih ada pemberian vaksin yang tidak halal, setelah adanya putusan MA ini, maka kami akan menuntut dan melaporkan…'], [300233479, 'pulchritudenj', datetime.datetime(2022, 4, 21, 13, 52, 44), '@tubirfess I KNOW RIGHT?!!! Greget banget baca komen laki blg konspirasi, pemerintah nyari untung, hati2 jd zombie,… https://t.co/Pn8SfCExqK']

In [ ]:
tweet_text = pd.DataFrame(data=data, 
                    columns=["id", "user", "time", "text"])
tweet_text

,id,user,time,text
0,359329138,dindamedina,2022-04-21 13:53:05,Tadi pagi baru vaksin yang ketiga dan nggak ad...
1,446174007,Indra_zill,2022-04-21 13:53:05,@popienastiti1 Gaji udah dalam bentuk makanan ...
2,1451457302628765703,nb1957_,2022-04-21 13:53:05,RT @deehandhanin: ruu tpks disahkan. vaksin ka...
3,1021087094,NugroDwi,2022-04-21 13:52:58,"RT @GratisTerbaik: ""Jika masih ada pemberian v..."
4,300233479,pulchritudenj,2022-04-21 13:52:44,@tubirfess I KNOW RIGHT?!!! Greget banget baca...
...,...,...,...,...
9995,1085708885044879365,AannasNoah,2022-04-21 02:18:41,RT @KKMPutrajaya: Hari Raya bakal menjelang. R...
9996,1131901132928737280,pinkgurlsonmode,2022-04-21 02:18:41,orang2 anti vaksin kaya gituuu!!! https://t.co...
9997,3249537901,giraffehannie,2022-04-21 02:18:36,RT @tubirfess: 2beer! Tak kira semua org tau p...
9998,1396331301959921664,GinanjarTrisno,2022-04-21 02:18:36,Vaksin Booster dan Prokes Ketat Strategi Ampuh...


In [ ]:
tweet_text.to_csv('/content/drive/MyDrive/FYP/Data/NEW_vaksin_data.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/MyDrive/FYP/Data/NEW_vaksin_data.csv"
tweet_text = pd.read_csv(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Data Cleaning

In [ ]:
# delete irrelevent columns
del tweet_text['time']
del tweet_text['id']
del tweet_text['user']
del tweet_text['Unnamed: 0']

In [ ]:
# remove RT@
def remove_reply(text):
    text = re.sub("RT @\w+: ","",text)
    return text

tweet_text['text'] = tweet_text['text'].apply(lambda x: remove_reply(x))

# remove url links
tweet_text['text'] = tweet_text['text'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)

# remove newline symbol
tweet_text['text'] = tweet_text['text'].replace(r'\n',' ', regex=True)

# remove extra multiple spaces between words
tweet_text['text'] = tweet_text['text'].replace('\s+', ' ', regex=True)


In [ ]:
# split the text if there is capital letter
# e.g. DiaMakan change to Dia Makan
def split(text):

    text = " ".join([a for a in re.split('([A-Z][a-z]+)', text) if a])

    return text

tweet_text['text'] = tweet_text['text'].apply(lambda x: split(x))

In [ ]:
def cleaning(text):

    # remove tag
    text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ", text).split())
    
    # lower text
    text = "".join(token.lower() for token in text)

    # remove number
    text = re.sub(r"\d", " ", text)
    return text

tweet_text['text'] = tweet_text['text'].apply(lambda x: cleaning(x))

tweet_text = tweet_text.reset_index()

del tweet_text['index']

In [ ]:
# check whether have vaksin word
tweet_text = tweet_text[tweet_text['text'].fillna(0).str.contains("vaksin",na=False)] 

In [ ]:
tweet_text

,text
0,tadi pagi baru vaksin yang ketiga dan nggak ad...
1,gaji udah dalam bentuk makanan bergizi akomoda...
2,ruu tpks disahkan vaksin kanker serviks diwaji...
3,jika masih ada pemberian vaksin yang tidak hal...
5,banyak yg tanya soal vaksin hpv kepada saya ya...
...,...
10577,vaksin booster dan prokes ketat strategi ampuh...
10579,orang anti vaksin kaya gituuu
10580,beer tak kira semua org tau pentingnya vaksin...
10581,vaksin booster dan prokes ketat strategi ampuh...


# Spelling Correction

In [ ]:

#----------------------------------------------------------------------------------------------------------------------
tweet_text['text'] = tweet_text['text'].replace(r"(\w*)(hah|gag)(\w*)", " haha ", regex=True) #Convert variation of 'haha'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ahakz|ahsbahaba|ha+ha|shsjsjs)\b", " haha ", regex=True) #Convert variation of 'haha'
tweet_text['text'] = tweet_text['text'].replace(r"(\w*)(huh)(\w*)", " huhu ", regex=True) #Convert variation of 'huhu'
tweet_text['text'] = tweet_text['text'].replace(r"(\w*)(heh)(\w*)", " hehe ", regex=True) #Convert variation of 'hehe'
tweet_text['text'] = tweet_text['text'].replace(r"(\w*)(hoh)(\w*)", " hoho ", regex=True) #Convert variation of 'hoho'
tweet_text['text'] = tweet_text['text'].replace(r"(\w*)(hih|hikh|sksksksksks)(\w*)", " hihi ", regex=True) #Convert variation of 'hihi'
tweet_text['text'] = tweet_text['text'].replace(r"(\w*)(kahk|kakaka)(\w*)", " kahkah ", regex=True) #Convert variation of 'kahkah'
tweet_text['text'] = tweet_text['text'].replace(r"(\w*)(keke)(\w*)", " keke ", regex=True) #Convert variation of 'keke'
tweet_text['text'] = tweet_text['text'].replace(r"(\w*)(kihk)(\w*)", " kihkih ", regex=True) #Convert variation of 'kihkih'
tweet_text['text'] = tweet_text['text'].replace(r"\b(eleh)(\w*)", " eleh ", regex=True) #Convert variation of 'eleh'
tweet_text['text'] = tweet_text['text'].replace(r"(\w*)(wkwk)(\w*)", " wakawaka ", regex=True) #Convert variation of 'wakawaka'
tweet_text['text'] = tweet_text['text'].replace(r"(alolo)(\w*)", " alolo ", regex=True) #Convert variation of 'alolo'
tweet_text['text'] = tweet_text['text'].replace(r"\b(aish+|ish+|eish)\b", " cis ", regex=True) #Convert variation of 'cis'
tweet_text['text'] = tweet_text['text'].replace(r"\b(arg+h|argh+|a+rgh|arh+|urgh)\b", " ah ", regex=True) #Convert variation of 'ah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pew|pfts|prft)\b", " ah ", regex=True) #Convert variation of 'fuh'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ddk)\b", " duduk ", regex=True) #Convert variation of 'duduk'
#----------------------------------------------------------------------------------------------------------------------
tweet_text['text'] = tweet_text['text'].replace(r"a+\b", "a", regex=True) #Remove multiple 'a' to a single 'a'
tweet_text['text'] = tweet_text['text'].replace(r"\ba+", "a", regex=True) #Remove multiple 'a' to a single 'a'
tweet_text['text'] = tweet_text['text'].replace(r"b+", "b", regex=True) #Remove multiple 'b' to a single 'b'
tweet_text['text'] = tweet_text['text'].replace(r"c+", "c", regex=True) #Remove multiple 'c' to a single 'c'
tweet_text['text'] = tweet_text['text'].replace(r"d+", "d", regex=True) #Remove multiple 'd' to a single 'd'
tweet_text['text'] = tweet_text['text'].replace(r"e+\b", "e", regex=True) #Remove multiple 'e' to a single 'e'
tweet_text['text'] = tweet_text['text'].replace(r"f+\b", "f", regex=True) #Remove multiple 'f' to a single 'f'
tweet_text['text'] = tweet_text['text'].replace(r"g+\b", "g", regex=True) #Remove multiple 'g' to a single 'g'
tweet_text['text'] = tweet_text['text'].replace(r"h+", "h", regex=True) #Remove multiple 'h' to a single 'h'
tweet_text['text'] = tweet_text['text'].replace(r"i+", "i", regex=True) #Remove multiple 'i' to a single 'i'
tweet_text['text'] = tweet_text['text'].replace(r"j+", "j", regex=True) #Remove multiple 'j' to a single 'j'
tweet_text['text'] = tweet_text['text'].replace(r"k+", "k", regex=True) #Remove multiple 'k' to a single 'k'
tweet_text['text'] = tweet_text['text'].replace(r"ll*\b", "l", regex=True) #Remove multiple 'l' to a single 'l'
tweet_text['text'] = tweet_text['text'].replace(r"m+", "m", regex=True) #Remove multiple 'm' to a single 'm'
tweet_text['text'] = tweet_text['text'].replace(r"n+", "n", regex=True) #Remove multiple 'n' to a single 'n'
tweet_text['text'] = tweet_text['text'].replace(r"o+\b", "o", regex=True) #Remove multiple 'o' to a single 'o'
tweet_text['text'] = tweet_text['text'].replace(r"p+", "p", regex=True) #Remove multiple 'p' to a single 'p'
tweet_text['text'] = tweet_text['text'].replace(r"q+", "q", regex=True) #Remove multiple 'q' to a single 'q'
tweet_text['text'] = tweet_text['text'].replace(r"r+", "r", regex=True) #Remove multiple 'r' to a single 'r'
tweet_text['text'] = tweet_text['text'].replace(r"s+", "s", regex=True) #Remove multiple 's' to a single 's'
tweet_text['text'] = tweet_text['text'].replace(r"t+", "t", regex=True) #Remove multiple 't' to a single 't'
tweet_text['text'] = tweet_text['text'].replace(r"u+\b", "u", regex=True) #Remove multiple 'u' to a single 'u'
tweet_text['text'] = tweet_text['text'].replace(r"v+", "v", regex=True) #Remove multiple 'v' to a single 'v'
tweet_text['text'] = tweet_text['text'].replace(r"w+", "w", regex=True) #Remove multiple 'w' to a single 'w'
tweet_text['text'] = tweet_text['text'].replace(r"x+", "x", regex=True) #Remove multiple 'x' to a single 'x'
tweet_text['text'] = tweet_text['text'].replace(r"y+", "y", regex=True) #Remove multiple 'y' to a single 'y'
tweet_text['text'] = tweet_text['text'].replace(r"z+", "z", regex=True) #Remove multiple 'z' to a single 'z'
#----------------------------------------------------------------------------------------------------------------------
tweet_text['text'] = tweet_text['text'].replace(r"(?<!a)(?<!e)(?<!i)(?<!o)(?<!u)(?<!l)(?<!g)(lah|la|le|lo|lar|ler|lor|lh)\b", " la", regex=True) #Normalizing 'la'
tweet_text['text'] = tweet_text['text'].replace(r"(?<!ta)(?<!ha)(?<!pu)(?<!dit)(nya|nye|nyo|nyer)\b", " nya", regex=True) #Normalizing 'nya'
tweet_text['text'] = tweet_text['text'].replace(r"(?<!ta)(?<!ha)(?<!pu)(?<!dit)nya\b", "", regex=True) #Remove 'nya'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tny|tnyk|taye|taya|tanya|tanye|tanyo|tanyer)\b", " tanya ", regex=True) #Convert variation of 'tanya'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ditnya)\b", " ditanya ", regex=True) #Convert variation of 'ditanya'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pny|punya|punye|punyo|punyer)\b", " punya ", regex=True) #Convert variation of 'punya'
tweet_text['text'] = tweet_text['text'].replace(r"\b(x)", "tak\n", regex=True) #Replace 'x' with 'tak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tk)\b", " tak ", regex=True) #Convert variation of 'tak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tk)(?!a)(?!e)(?!at)(?!jut)", " tak", regex=True) #Convert variation of 'tak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tak)(?!ut)(?!jut)(?!ejut)(?!ot)(?!raw)(?!at)(?!sub)(?!tik)", " tak\n ", regex=True) #Convert variation of 'tak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(eluar)\b", " keluar ", regex=True) #Convert variation of 'keluar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(k|okay|okei|oke|kei|okeh)\b", " ok ", regex=True) #Convert variation of 'ok'
tweet_text['text'] = tweet_text['text'].replace(r"\b(hei|hey)\b", " hai ", regex=True) #Convert variation of 'hai'
tweet_text['text'] = tweet_text['text'].replace(r"\b(eyh|ey|ei|e+h)\b", " eh ", regex=True) #Conveelokrt variation of 'eh'
tweet_text['text'] = tweet_text['text'].replace(r"\b(wey|weyh|we+i)\b", " weh ", regex=True) #Convert variation of 'weh'
tweet_text['text'] = tweet_text['text'].replace(r"\b(weiaku)\b", " weh aku ", regex=True) #Convert variation of 'weh aku'
tweet_text['text'] = tweet_text['text'].replace(r"\b(wow|woah)\b", " wah ", regex=True) #Convert variation of 'wah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(wo+i|woit|woy)\b", " woi ", regex=True) #Convert variation of 'woi'
tweet_text['text'] = tweet_text['text'].replace(r"\b(hk|huk|ho)\b", " hok ", regex=True) #Convert variation of 'hok'
#----------------------------------------------------------------------------------------------------------------------
tweet_text['text'] = tweet_text['text'].replace(r"\b(mlayu|mlyu)\b", " melayu ", regex=True) #Convert variation of 'melayu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(jhr|jhor|joho|johor)\b", " johor ", regex=True) #Convert variation of 'johor'
tweet_text['text'] = tweet_text['text'].replace(r"\b(msa|msia|mlysia|mlysa|malaysia|msia)\b", " malaysia ", regex=True) #Convert variation of 'malaysia'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kelate|klate|klatey|kltn|kelantan)\b", " kelantan ", regex=True) #Convert variation of 'kelantan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tranung|tranu|tegganu|terenganu|ganu|teganu)\b", " terengganu ", regex=True) #Convert variation of 'terengganu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kl|keyel|kolumpo|kolumpor)\b", " kuala lumpur ", regex=True) #Convert variation of 'kuala lumpur'
tweet_text['text'] = tweet_text['text'].replace(r"\b(indon|indo)\b", " indonesia ", regex=True) #Convert variation of 'indonesia'
tweet_text['text'] = tweet_text['text'].replace(r"\b(rm|hengget|hinggit)\b", " ringgit\nmalaysia ", regex=True) #Convert variation of 'ringgit malaysia'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ig|insta|instagram|instgrm)\b", " instagram ", regex=True) #Convert variation of 'instagram'
tweet_text['text'] = tweet_text['text'].replace(r"\b(fb|fesbuk|facebook|pesbuk)\b", " facebook ", regex=True) #Convert variation of 'facebook'
tweet_text['text'] = tweet_text['text'].replace(r"\b(socmed)\b", " social media ", regex=True) #Convert variation of 'social media'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ws|wasap|whtsap|whatsap|whtsp|wasep|wsap)\b", " whatsapp ", regex=True) #Convert variation of 'whatsapp'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bhsa|bahsa|bahaso)\b", " bahasa ", regex=True) #Convert variation of 'bahasa'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bi)\b", " bahasa inggeris ", regex=True) #Convert variation of 'bahasa inggeris'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bm)\b", " bahasa melayu ", regex=True) #Convert variation of 'bahasa melayu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tv|tb)\b", " televisyen ", regex=True) #Convert variation of 'televisyen'
tweet_text['text'] = tweet_text['text'].replace(r"\b(gomen|govt|gov)\b", " kerajaan ", regex=True) #Convert variation of 'kerajaan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(uni)\b", " universiti ", regex=True) #Convert variation of 'universiti'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kg|kmpng|kampong|kmpong|kmpung|kpg|kampuang)\b", " kampung ", regex=True) #Convert variation of 'kampung'
tweet_text['text'] = tweet_text['text'].replace(r"\b(iv)\b", " temu duga ", regex=True) #Convert variation of 'temu duga'
tweet_text['text'] = tweet_text['text'].replace(r"\b(fav)\b", " kegemaran ", regex=True) #Convert variation of 'kegemaran'
tweet_text['text'] = tweet_text['text'].replace(r"\b(yt)\b", " youtube ", regex=True) #Convert variation of 'youtube'
tweet_text['text'] = tweet_text['text'].replace(r"\b(twt|twit)\b", " tweet ", regex=True) #Convert variation of 'tweet'
tweet_text['text'] = tweet_text['text'].replace(r"\b(rt)\b", " retweet ", regex=True) #Convert variation of 'retweet'
tweet_text['text'] = tweet_text['text'].replace(r"\b(jr)\b", " junior ", regex=True) #Convert variation of 'junior'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bro|brader|brothers)\b", " brother ", regex=True) #Convert variation of 'brother'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sis|sistur|sisters)\b", " sister ", regex=True) #Convert variation of 'sister'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dm)\b", " direct message  ", regex=True) #Convert variation of 'direct message'
tweet_text['text'] = tweet_text['text'].replace(r"\b(otw)\b", " on the way ", regex=True) #Convert variation of 'on the way'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pls|plis)\b", " please ", regex=True) #Convert variation of 'please'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pleasela)\b", " please la ", regex=True) #Convert variation of 'please la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bf|boyfren)\b", " boyfriend ", regex=True) #Convert variation of 'boyfriend'
tweet_text['text'] = tweet_text['text'].replace(r"\b(gf)\b", " girlfriend ", regex=True) #Convert variation of 'girlfriend'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mcd|mekdi)\b", " mcdonald ", regex=True) #Convert variation of 'mcdonald'
tweet_text['text'] = tweet_text['text'].replace(r"\b(twiter)\b", " twitter ", regex=True) #Convert variation of 'twitter'
#----------------------------------------------------------------------------------------------------------------------
tweet_text['text'] = tweet_text['text'].replace(r"\b(allahu|allahuakbar)\b", " allahuakbar ", regex=True) #Convert variation of 'allahuakbar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(salamualaikum|salam|asalam|assalam|asslmlkm|assmekum|aslmlkm|asmekum|semekum|smekum)\b", " assalamualaikum ", regex=True) #Convert variation of 'assalamualaikum'
tweet_text['text'] = tweet_text['text'].replace(r"\b(wasalamu|wassalamu|wasalam|wassalam|wsslm|wsslam|wslm|wslam)\b", " waalaikumsalam ", regex=True) #Convert variation of 'waalaikumsalam'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ame+n)\b", " amin ", regex=True) #Convert variation of 'amin'
tweet_text['text'] = tweet_text['text'].replace(r"\b(redha|redho|redo)\b", " reda ", regex=True) #Convert variation of 'reda'
tweet_text['text'] = tweet_text['text'].replace(r"\b(wudu|wuduq|wudo|wudoq|wudhu)\b", " wuduk ", regex=True) #Convert variation of 'wuduk'
tweet_text['text'] = tweet_text['text'].replace(r"\b(qiyamulail|qiyamullail)\b", " qiamullail ", regex=True) #Convert variation of 'qiamullail'
tweet_text['text'] = tweet_text['text'].replace(r"\b(inshaallah)\b", " insyaallah ", regex=True) #Convert variation of 'insyaallah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dhuha|dhuho|duho)\b", " duha ", regex=True) #Convert variation of 'duha'
tweet_text['text'] = tweet_text['text'].replace(r"\b(alhamdulilah)\b", " alhamdulillah ", regex=True) #Convert variation of 'alhamdulillah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(yarabi)\b", " ya rabbi ", regex=True) #Convert variation of 'ya rabbi'
tweet_text['text'] = tweet_text['text'].replace(r"\b(yamat)\b", " ya amat ", regex=True) #Convert variation of 'ya amat'
#----------------------------------------------------------------------------------------------------------------------
tweet_text['text'] = tweet_text['text'].replace(r"\b(harey|harem|hare)\b", " haram ", regex=True) #Convert variation of 'haram'
tweet_text['text'] = tweet_text['text'].replace(r"\b(hg|hanq|ang)\b", " hang ", regex=True) #Convert variation of 'hang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(hgpa|hampa|hmpa|ampa)\b", " hangpa ", regex=True) #Convert variation of 'hangpa'
tweet_text['text'] = tweet_text['text'].replace(r"\b(abe|abg|abng|bang)\b", " abang ", regex=True) #Convert variation of 'abang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(awok|awk)\b", " awak ", regex=True) #Convert variation of 'awak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kak|akak|akok|akk|kakok)\b", " kakak ", regex=True) #Convert variation of 'kakak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(adek|dik)\b", " adik ", regex=True) #Convert variation of 'adik'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sy|sya|sye|sayo|saye)\b", " saya ", regex=True) #Convert variation of 'saya'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dio|die|dya|dy|dye|diak)\b", " dia ", regex=True) #Convert variation of 'dia'
tweet_text['text'] = tweet_text['text'].replace(r"\b(diorang|diaorang|dorang|drang|dorg|diorg|dorng|diaorg|diorng|dyorg)\b", " dia orang ", regex=True) #Convert variation of 'dia orang'
tweet_text['text'] = tweet_text['text'].replace(r"(korang|kauorang|kaurang|korg|korng|korunk)\b", " kau orang ", regex=True) #Convert variation of 'kau orang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kitorang|kiteorang|ktaorang|kteorang|kitorg|kiteorg|ktaorg|kteorg)\b", " kita orang ", regex=True) #Convert variation of 'kita orang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kaj|kaw|ekau|engko|engkau)\b", " kau ", regex=True) #Convert variation of 'kau'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kito|kite|kta|kte|titew|ktk|kitak|kitok|kiter)\b", " kita ", regex=True) #Convert variation of 'kita'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mreka|mrka|mrke|mrk)\b", " mereka ", regex=True) #Convert variation of 'mereka'
tweet_text['text'] = tweet_text['text'].replace(r"\b(deme|demo)\b", " mereka ", regex=True) #Convert variation of 'dema'
tweet_text['text'] = tweet_text['text'].replace(r"\b(org|ore|orng|urang|ong|oqang|urg)\b", " orang ", regex=True) #Convert variation of 'orang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(semorang)\b", " semua orang ", regex=True) #Convert variation of 'semua orang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(gini|bagini|bgni|gni|begini|bgini|sbegini|gining)\b", " begini ", regex=True) #Convert variation of 'begini'
tweet_text['text'] = tweet_text['text'].replace(r"\b(beginilah)\b", " begini la ", regex=True) #Convert variation of 'begini la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ado+h|adoi+|adui+|adu+h|haduh|adeh)\b", " aduh ", regex=True) #Convert variation of 'aduh'
tweet_text['text'] = tweet_text['text'].replace(r"\b(aduyai|aduy)\b", " aduhai ", regex=True) #Convert variation of 'aduhai'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kmudian|kemudia)\b", " kemudian ", regex=True) #Convert variation of 'kemudian'
tweet_text['text'] = tweet_text['text'].replace(r"\b(walaupon|wlaupun|wlaupon|wlupun|wpun|wlupn|wlpn)\b", " walaupun ", regex=True) #Convert variation of 'walaupun'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ttp|ttpi|tetapi)\b", " tetapi ", regex=True) #Convert variation of 'tetapi'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tap|tetap)\b", " tetap ", regex=True) #Convert variation of 'tetap'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tntu|tentu)\b", " tentu ", regex=True) #Convert variation of 'tentu'
tweet_text['text'] = tweet_text['text'].replace(r"(mggu)\b", " minggu ", regex=True) #Convert variation of 'minggu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ckgu|cgu|cg)\b", " cikgu ", regex=True) #Convert variation of 'cikgu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mpos|mampos|mapuh|mampoih|mampuih)\b", " mampus ", regex=True) #Convert variation of 'mampus'
tweet_text['text'] = tweet_text['text'].replace(r"\b(beratuq|bratoq|bratur|brator|beratu)\b", " beratur ", regex=True) #Convert variation of 'beratur'
tweet_text['text'] = tweet_text['text'].replace(r"\b(psr|pasor|paso|psar)\b", " pasar ", regex=True) #Convert variation of 'pasar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(klako|klakar|kelaka)\b", " kelakar ", regex=True) #Convert variation of 'kelakar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sluar|slua|seluaq|seluo|seluor)\b", " seluar ", regex=True) #Convert variation of 'seluar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bru|br)\b", " baru ", regex=True) #Convert variation of 'baru'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bharu|bahru)\b", " baharu ", regex=True) #Convert variation of 'baharu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bhg)\b", " bahagian ", regex=True) #Convert variation of 'bahagian'
tweet_text['text'] = tweet_text['text'].replace(r"\b(biadap)\b", " biadab ", regex=True) #Convert variation of 'biadab'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mbr|membe|mber|mbe|memba)\b", " member ", regex=True) #Convert variation of 'member'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kmptr|kmputer)\b", " komputer ", regex=True) #Convert variation of 'komputer'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bbir|bir)\b", " bibir ", regex=True) #Convert variation of 'bibir'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kapir)\b", " kafir ", regex=True) #Convert variation of 'kafir'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mnusio|manusio|mnusia|menasio)\b", " manusia ", regex=True) #Convert variation of 'manusia'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bko|bako|bakor|bkar|bkr|bakaq)\b", " bakar ", regex=True) #Convert variation of 'bakar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(idup|hdup|idop)\b", " hidup ", regex=True) #Convert variation of 'hidup'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kehidupe)\b", " kehidupan ", regex=True) #Convert variation of 'kehidupan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(hrp|hrap)\b", " harap ", regex=True) #Convert variation of 'harap'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mjlis|mjls)\b", " majlis ", regex=True) #Convert variation of 'majlis'
tweet_text['text'] = tweet_text['text'].replace(r"\b(hadith|hdis)\b", " hadis ", regex=True) #Convert variation of 'hadis'
tweet_text['text'] = tweet_text['text'].replace(r"\b(jnis|jns|enis|jenih|pesen|peseng)\b", " jenis ", regex=True) #Convert variation of 'jenis'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tg|tntg|tntng|tentang)\b", " tentang ", regex=True) #Convert variation of 'tentang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sorg|sorang|surang|sensorg|sorng|sowg|sowng|seorey|sensorang)\b", " seorang ", regex=True) #Convert variation of 'seorang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tiap|stiap)\b", " setiap ", regex=True) #Convert variation of 'setiap'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sasudah|udah|suda|sudag|sudoh|kesudah|udoh|udh|sdh|dah)\b", " sudah ", regex=True) #Convert variation of 'sudah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(plg|pling|plng)\b", " paling ", regex=True) #Convert variation of 'paling'
tweet_text['text'] = tweet_text['text'].replace(r"\b(jak|kejak|jap|kjp|jp|stg|sat|st)\b", " kejap ", regex=True) #Convert variation of 'kejap'
tweet_text['text'] = tweet_text['text'].replace(r"\b(satni)\b", " kejap ini ", regex=True) #Convert variation of 'kejap ini'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ekgi|ekgy|jakgi|japgi|kjpgi|jpgi|kjaplgi|satgi)\b", " kejap lagi ", regex=True) #Convert variation of 'kejap lagi'
tweet_text['text'] = tweet_text['text'].replace(r"\b(skrg|skang|skarang|skrang|skrng|skg|sekrg|sekrng)", " sekarang ", regex=True) #Convert variation of 'sekarang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(slagi|slg|slgi)\b", " selagi ", regex=True) #Convert variation of 'selagi'
tweet_text['text'] = tweet_text['text'].replace(r"\b(slalu|sllu|salu|selaluh)\b", " selalu ", regex=True) #Convert variation of 'selalu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(agk|agok)\b", " agak ", regex=True) #Convert variation of 'agak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(asl|asai|asal)", " asal ", regex=True) #Convert variation of 'asal'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pd|pda|pado)\b", " pada ", regex=True) #Convert variation of 'pada'
tweet_text['text'] = tweet_text['text'].replace(r"\b(awt)\b", " awat ", regex=True) #Convert variation of 'awat'
tweet_text['text'] = tweet_text['text'].replace(r"\b(drpd|drpda|dripada|daripado|dpda)\b", " daripada ", regex=True) #Convert variation of 'daripada'
tweet_text['text'] = tweet_text['text'].replace(r"\b(atah|ataih|ats)\b", " atas ", regex=True) #Convert variation of 'atas'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bowoh|bwh)\b", " bawah ", regex=True) #Convert variation of 'bawah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(guane|guano|gane|gano)\b", " bagaimana ", regex=True) #Convert variation of 'bagaimana'
tweet_text['text'] = tweet_text['text'].replace(r"\b(akn)\b", " akan ", regex=True) #Convert variation of 'akan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dpn|dpan|depn)\b", " depan ", regex=True) #Convert variation of 'depan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sguh|sngh|sgguh|sesungguh)\b", " sungguh ", regex=True) #Convert variation of 'sungguh'
tweet_text['text'] = tweet_text['text'].replace(r"\b(perna|pernh|prnh|penah|pnh)\b", " pernah ", regex=True) #Convert variation of 'pernah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kli|kalix)\b", " kali ", regex=True) #Convert variation of 'kali'
tweet_text['text'] = tweet_text['text'].replace(r"\b(rmai|rame)\b", " ramai ", regex=True) #Convert variation of 'ramai'
tweet_text['text'] = tweet_text['text'].replace(r"\b(gtau|gtaw|bagitaw|gitaw|bgitaw|habaq|bgthu|bgtahu|bagitau|bgtau|oyak+|royak|broyak|beroyak|ghoyak|royat)\b", " beritahu ", regex=True) #Convert variation of 'beritahu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(skali|skli|skalih|sesekali)\b", " sekali ", regex=True) #Convert variation of 'sekali'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sape|spa|spe|sapo)\b", " siapa ", regex=True) #Convert variation of 'siapa'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dn)\b", " dan ", regex=True) #Convert variation of 'dan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dri|dr)\b", " dari ", regex=True) #Convert variation of 'dari'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dlm|dale|dalan|dlam|lam)\b", " dalam ", regex=True) #Convert variation of 'dalam'
tweet_text['text'] = tweet_text['text'].replace(r"\b(cume|cumo|cme|cma)\b", " cuma ", regex=True) #Convert variation of 'cuma'
tweet_text['text'] = tweet_text['text'].replace(r"\b(cth|cntoh|cntuh)\b", " contoh ", regex=True) #Convert variation of 'contoh'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kluar|kuar|kuor|kluaq|klua|keluaq|keluar|keluo)\b", " keluar ", regex=True) #Convert variation of 'keluar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bawak|bwak|bawok|bwk|bwa|bowok|baok)", " bawa ", regex=True) #Convert variation of 'bawa'
tweet_text['text'] = tweet_text['text'].replace(r"\b(jugak|gak|jgk|jgok|jgak|jugok|jugaq|juo|jga|gk|gok|jg|jua)\b", " juga ", regex=True) #Convert variation of 'juga'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ari|hri|hr|ri|heri)\b", " hari ", regex=True) #Convert variation of 'hari'
tweet_text['text'] = tweet_text['text'].replace(r"\b(besok|esk|besuk|sok|isok)\b", " esok ", regex=True) #Convert variation of 'esok'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pg|pgi)\b", " pagi ", regex=True) #Convert variation of 'pagi'
tweet_text['text'] = tweet_text['text'].replace(r"(hrmat)\b", " hormat ", regex=True) #Convert variation of 'hormat'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kok|rkok|rk)\b", " rokok ", regex=True) #Convert variation of 'rokok'
tweet_text['text'] = tweet_text['text'].replace(r"\b(hargo|hrgo|hrga|harga|hrg|harago)\b", " harga ", regex=True) #Convert variation of 'harga'
tweet_text['text'] = tweet_text['text'].replace(r"\b(umo|umoq|umr|umur)\b", " umur ", regex=True) #Convert variation of 'umur'
tweet_text['text'] = tweet_text['text'].replace(r"\b(muto|moto|mtr|motor|mtor|motosikal)\b", " motosikal ", regex=True) #Convert variation of 'motosikal'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tmpat|tmpt|tempat)\b", " tempat ", regex=True) #Convert variation of 'tempat'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tfon|tepon|tepun|tlfn|tlfon|tipon|telefon|fon)\b", " telefon ", regex=True) #Convert variation of 'telefon'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tndas|tnds)\b", " tandas ", regex=True) #Convert variation of 'tandas'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kwn|kawe|kwan|kawn|kengkawan|kekawan)\b", " kawan ", regex=True) #Convert variation of 'kawan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(aiyaq|ayor|ayer|ayaq)\b", " air ", regex=True) #Convert variation of 'air'
tweet_text['text'] = tweet_text['text'].replace(r"(baek|bek|bebaik|baik)\b", " baik ", regex=True) #Convert variation of 'baik'
tweet_text['text'] = tweet_text['text'].replace(r"\b(rmah|rmh|rumh|umah|dumoh|rumoh|umh|umoh|qhumah|ghumah)\b", " rumah ", regex=True) #Convert variation of 'rumah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(comei|comey)\b", " comel ", regex=True) #Convert variation of 'comel'
tweet_text['text'] = tweet_text['text'].replace(r"\b(setan|syaiton|syaithon|sete)\b", " syaitan ", regex=True) #Convert variation of 'syaitan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(peranga|prngai|pranga|prangai|parangai)\b", " perangai ", regex=True) #Convert variation of 'perangai'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kawen|kawin|khwin|kwen)", " kahwin\n ", regex=True) #Convert variation of 'kahwin'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ank|anok)", " anak ", regex=True) #Convert variation of 'anak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(anaku|anak+u)", " anak aku ", regex=True) #Convert variation of 'anak aku'
tweet_text['text'] = tweet_text['text'].replace(r"\b(semngt|smangat|smngat)\b", " semangat ", regex=True) #Convert variation of 'semangat'
tweet_text['text'] = tweet_text['text'].replace(r"\b(cti|coti)\b", " cuti ", regex=True) #Convert variation of 'cuti'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sejid|msjd|msjid|masjd)\b", " masjid ", regex=True) #Convert variation of 'masjid'
tweet_text['text'] = tweet_text['text'].replace(r"\b(brg|brng|brang)\b", " barang ", regex=True) #Convert variation of 'barang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(meroh)\b", " merah ", regex=True) #Convert variation of 'merah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(slisih)\b", " selisih ", regex=True) #Convert variation of 'selisih'
tweet_text['text'] = tweet_text['text'].replace(r"\b(slmt|slamat|slmt)\b", " selamat ", regex=True) #Convert variation of 'selamat'
tweet_text['text'] = tweet_text['text'].replace(r"\b(anjir|anj|njing)\b", " anjing ", regex=True) #Convert variation of 'anjing'
tweet_text['text'] = tweet_text['text'].replace(r"\b(zmn|zman|jaman)\b", " zaman ", regex=True) #Convert variation of 'zaman'
tweet_text['text'] = tweet_text['text'].replace(r"\b(besa|besaq|besau|besor|beso)\b", " besar ", regex=True) #Convert variation of 'besar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(knl|knal|kenai)\b", " kenal ", regex=True) #Convert variation of 'kenal'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pnting|pntng)\b", " penting ", regex=True) #Convert variation of 'penting'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sbar|sbr|sabor|sabo|sabaq|saba|sobar)\b", " sabar ", regex=True) #Convert variation of 'sabar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(syg|syang|sayng|ayang)\b", " sayang ", regex=True) #Convert variation of 'sayang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(awai|awl)\b", " awal ", regex=True) #Convert variation of 'awal'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mlam|mlm|male)\b", " malam ", regex=True) #Convert variation of 'malam'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pndai|pndi|pandei|pandey|pnda)\b", " pandai ", regex=True) #Convert variation of 'pandai'
tweet_text['text'] = tweet_text['text'].replace(r"\b(crita|cita|cite|cito|citer|crite|ceqita|ceghita|cerite)\b", " cerita ", regex=True) #Convert variation of 'cerita'
tweet_text['text'] = tweet_text['text'].replace(r"\b(trus|truih|trs|tros|teroih|teruih|teruh)\b", " terus ", regex=True) #Convert variation of 'terus'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dtg|date|dtang|datg|datng)\b", " datang ", regex=True) #Convert variation of 'datang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mls|mlas|mals|meleis|maleh|males|malaih)\b", " malas ", regex=True) #Convert variation of 'malas'
tweet_text['text'] = tweet_text['text'].replace(r"\b(thn|taun|taon|thun)\b", " tahun ", regex=True) #Convert variation of 'tahun'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tgh|tngh|tngah|tgah|tengh|tetengah|tangah)\b", " tengah ", regex=True) #Convert variation of 'tengah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(igt|ingt|ngat|hingat|ingak)\b", " ingat ", regex=True) #Convert variation of 'ingat'
tweet_text['text'] = tweet_text['text'].replace(r"\b(igtkan|ingtkan|ngatkan)\b", " ingatkan ", regex=True) #Convert variation of 'ingatkan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(lame|lamo|lma|lme|lmer|lelama|lamalama|llma|lm)\b", " lama ", regex=True) #Convert variation of 'lama'
tweet_text['text'] = tweet_text['text'].replace(r"\b(naek)\b", " naik ", regex=True) #Convert variation of 'naik'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pnt|pnat|ponat)\b", " penat ", regex=True) #Convert variation of 'penat'
tweet_text['text'] = tweet_text['text'].replace(r"\b(gmba|gmbar|gmbr|gambo|gambaq|gamba)\b", " gambar ", regex=True) #Convert variation of 'gambar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pitih|duiet|duet)\b", " duit ", regex=True) #Convert variation of 'duit'
tweet_text['text'] = tweet_text['text'].replace(r"\b(snyum|snym|senyun)\b", " senyum ", regex=True) #Convert variation of 'senyum'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dap|sodap|sdp|sdap)\b", " sedap ", regex=True) #Convert variation of 'sedap'
tweet_text['text'] = tweet_text['text'].replace(r"\b(same|sma|sme|smer|samo|sesama|sm)\b", " sama ", regex=True) #Convert variation of 'sama'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sgt|sgat|sngat|sngt|ngak)\b", " sangat ", regex=True) #Convert variation of 'sangat'
tweet_text['text'] = tweet_text['text'].replace(r"\b(keje|kijo|kejew|kerija|kerajo|kojo|keja|kije|krja|karajo|kerje|kje)\b", " kerja ", regex=True) #Convert variation of 'kerja'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sket|skit|sikik|siket|cikit)\b", " sikit ", regex=True) #Convert variation of 'sikit'
tweet_text['text'] = tweet_text['text'].replace(r"\b(cukop|ckup|ckop)\b", " cukup ", regex=True) #Convert variation of 'cukup'
tweet_text['text'] = tweet_text['text'].replace(r"\b(cba|cubo|cbo|coba|cer)\b", " cuba ", regex=True) #Convert variation of 'cuba'
tweet_text['text'] = tweet_text['text'].replace(r"\b(cubala|cubolah)\b", " cuba la ", regex=True) #Convert variation of 'cuba la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(cntik|cntk|cntx)\b", " cantik ", regex=True) #Convert variation of 'cantik'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pnjg|pnjang|pnjng|panje)\b", " panjang ", regex=True) #Convert variation of 'panjang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(jd|jdi)\b", " jadi ", regex=True) #Convert variation of 'jadi'
tweet_text['text'] = tweet_text['text'].replace(r"\b(jdlah|jdilah|jadilah|jdlh|jdilh|jadilh|jdla|jdila|jadila)\b", " jadi la ", regex=True) #Convert variation of 'jadi la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(menjdk)\b", " menjadi ", regex=True) #Convert variation of 'menjadi'
tweet_text['text'] = tweet_text['text'].replace(r"\b(len|laen|laing)\b", " lain ", regex=True) #Convert variation of 'lain'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dlu|dolu|dulu|duluh)\b", " dulu ", regex=True) #Convert variation of 'dulu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kate|kater|kato|katanya)\b", " kata ", regex=True) #Convert variation of 'kata'
tweet_text['text'] = tweet_text['text'].replace(r"\b(suko|suker|suke|suki)\b", " suka ", regex=True) #Convert variation of 'suka'
tweet_text['text'] = tweet_text['text'].replace(r"(kejuk|kjut|kzut|kezut|kejot|kejuit)\b", "kejut ", regex=True) #Convert variation of 'kejut'
tweet_text['text'] = tweet_text['text'].replace(r"(jtuh|jatoh|jth)\b", " jatuh ", regex=True) #Convert variation of 'jatuh'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kene|kne|gena|kna|kena|kono|keno|kanai|kenak|pekena)\b", " kena ", regex=True) #Convert variation of 'kena'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bekki|beki)\b", " baiki ", regex=True) #Convert variation of 'baiki'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pelan)\b", " perlahan ", regex=True) #Convert variation of 'perlahan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(trun|trn|turun)\b", " turun ", regex=True) #Convert variation of 'turun'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tukaq|tka|tkr|tukar|tuko|tuka)\b", " tukar ", regex=True) #Convert variation of 'tukar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tgu|tggu|tunggu)\b", " tunggu ", regex=True) #Convert variation of 'tunggu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tido|tidoq|tidok|tdo|tidur)\b", " tidur ", regex=True) #Convert variation of 'tidur'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tetido|tertido)", " tertidur ", regex=True) #Convert variation of 'tertidur'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tnggy|tnggal|tinggai|tingglkn|tggai|tinggal|tinggey|tingga)\b", " tinggal ", regex=True) #Convert variation of 'tinggal'
tweet_text['text'] = tweet_text['text'].replace(r"\b(isap|hisap)\b", " hisap ", regex=True) #Convert variation of 'hisap'
tweet_text['text'] = tweet_text['text'].replace(r"\b(trimo|trima|terimo|terima|treme|terime|nerimak)\b", " terima ", regex=True) #Convert variation of 'terima'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tgur|tegoq|teguq|tegur)\b", " tegur ", regex=True) #Convert variation of 'tegur'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tnggung)\b", " tanggung ", regex=True) #Convert variation of 'tanggung'
tweet_text['text'] = tweet_text['text'].replace(r"\b(jwb|jwab)\b", " jawab ", regex=True) #Convert variation of 'jawab'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tggjawab|tggjwb|tgjawab)\b", " tanggungjawab ", regex=True) #Convert variation of 'tanggungjawab'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tngkap|tngkp|tgkp|tangkap)\b", " tangkap ", regex=True) #Convert variation of 'tangkap' 
tweet_text['text'] = tweet_text['text'].replace(r"\b(tnp|tnpa|tanpa)\b", " tanpa ", regex=True) #Convert variation of 'tanpa'
tweet_text['text'] = tweet_text['text'].replace(r"(takot|tkot|tkut|tkt)\b", " takut ", regex=True) #Convert variation of 'takut'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tampaq|tampor|tmpr)\b", " tampar ", regex=True) #Convert variation of 'tampar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pggil|pnggil|pgil|pangil)\b", " panggil ", regex=True) #Convert variation of 'panggil'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bace|baco)\b", " baca ", regex=True) #Convert variation of 'baca'
tweet_text['text'] = tweet_text['text'].replace(r"\b(byar|byr|bayo|bayor|bayaq)\b", " bayar ", regex=True) #Convert variation of 'bayar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(lyan|lyn)\b", " layan ", regex=True) #Convert variation of 'layan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sgup|sngp|sngop|sangop|sggup)\b", " sanggup ", regex=True) #Convert variation of 'sanggup'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mrah|maroh|mrh)\b", " marah ", regex=True) #Convert variation of 'marah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(msak|msk)\b", " masak ", regex=True) #Convert variation of 'masak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ajk|ajok)\b", " ajak ", regex=True) #Convert variation of 'ajak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(aja|ajaar|ajr|ajaq)\b", " ajar ", regex=True) #Convert variation of 'ajar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(cpt|cpat|capat|pat)\b", " cepat ", regex=True) #Convert variation of 'cepat'
tweet_text['text'] = tweet_text['text'].replace(r"\b(anta|hntr|hanto|anto|hantor|hntaq|hantaq|hantar)\b", " hantar ", regex=True) #Convert variation of 'hantar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(antakan)\b", " hantarkan ", regex=True) #Convert variation of 'hantarkan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(nakhantaq)\b", " nak hantar ", regex=True) #Convert variation of 'nak hantar '
tweet_text['text'] = tweet_text['text'].replace(r"\b(baucer)\b", " baucar ", regex=True) #Convert variation of ' baucar '
tweet_text['text'] = tweet_text['text'].replace(r"\b(gune|guno|gna|gne)\b", " guna ", regex=True) #Convert variation of 'guna'
tweet_text['text'] = tweet_text['text'].replace(r"(abes|abis|abih|habeh|habih|hbes|hbis|h+abis|hbs)\b", "habis ", regex=True) #Convert variation of 'habis'
tweet_text['text'] = tweet_text['text'].replace(r"\b(hbeskan|hbeskn)\b", " habiskan ", regex=True) #Convert variation of 'habiskan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ikot)\b", " ikut ", regex=True) #Convert variation of 'ikut'
tweet_text['text'] = tweet_text['text'].replace(r"\b(napok|nmpk|nmpak|nampok)\b", " nampak ", regex=True) #Convert variation of 'nampak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sruh|suh|suruah)\b", " suruh ", regex=True) #Convert variation of 'suruh'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dsruh|dsuruah|dsruah|disuruah)\b", " di suruh ", regex=True) #Convert variation of 'di suruh'
tweet_text['text'] = tweet_text['text'].replace(r"\b(gado|gadoh|gdh)\b", " gaduh ", regex=True) #Convert variation of 'gaduh'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kul|pkul|pkl|pkol|pukol|pukoi|kol)\b", " pukul ", regex=True) #Convert variation of 'pukul'
tweet_text['text'] = tweet_text['text'].replace(r"\b(denga|dengaq|dengey|dengo|dengor|dgor+|dgq|dgr|dngq)\b", " dengar ", regex=True) #Convert variation of 'dengar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(susoh|sah|sh|susa)\b", " susah ", regex=True) #Convert variation of 'susah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(cr|cri|carik)\b", " cari ", regex=True) #Convert variation of 'cari'
tweet_text['text'] = tweet_text['text'].replace(r"\b(betol|betui|tol|btol|btul|btui|btl|btoi|betoi|betu)\b", " betul ", regex=True) #Convert variation of 'betul'
tweet_text['text'] = tweet_text['text'].replace(r"\b(brapa|berapo|brpa|brp|bape)\b", " berapa ", regex=True) #Convert variation of 'berapa'
tweet_text['text'] = tweet_text['text'].replace(r"((?<!ruq)(?<!pa)(?<!kun)yah|(?<!pa)yoh|yh|pyh|pyah|payh|payoh)\b", " payah ", regex=True) #Convert variation of 'payah'
tweet_text['text'] = tweet_text['text'].replace(r"(blajaq|blajar|bljq|bljr|bljaq|bljar|blajo|belajaq|belajaq|blaja)", " belajar ", regex=True) #Convert variation of 'belajar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(amekaw)\b", " ambil kau ", regex=True) #Convert variation of 'ambil kau'
tweet_text['text'] = tweet_text['text'].replace(r"\b(amek|amik|ambik|ame|ambk)", " ambil ", regex=True) #Convert variation of 'ambil'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bnda|bendo|bnde|bende|menda|mende|mnde|mnda)\b", " benda ", regex=True) #Convert variation of 'benda'
tweet_text['text'] = tweet_text['text'].replace(r"\b(lom|lum|blm|blom|blum|belom)\b", " belum ", regex=True) #Convert variation of 'belum'
tweet_text['text'] = tweet_text['text'].replace(r"\b(blkg|blkng)\b", " belakang ", regex=True) #Convert variation of 'belakang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(blakey|belako|blaka|blako|blaker|belake|blake)\b", " belaka ", regex=True) #Convert variation of 'belaka'
tweet_text['text'] = tweet_text['text'].replace(r"\b(nape|knpa|knp|kenape|knape|knpe|knapa)\b", " kenapa ", regex=True) #Convert variation of 'kenapa'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pkai|pki|pake|paka)\b", " pakai ", regex=True) #Convert variation of 'pakai'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sndiri|sndri|sdiri|sediri|sindiri)\b", " sendiri ", regex=True) #Convert variation of 'sendiri'
tweet_text['text'] = tweet_text['text'].replace(r"\b(msuk|msk|masok|msok|masuak)\b", " masuk ", regex=True) #Convert variation of 'masuk'
tweet_text['text'] = tweet_text['text'].replace(r"(byk|bnyk|bnyak|bnyok|banyok)", " banyak ", regex=True) #Convert variation of 'banyak'
tweet_text['text'] = tweet_text['text'].replace(r"(kebanyaka|kebanyakkan|kebnyakan|kebnyakkan|kebnyakkn|kebnyakn|kbanyakan)", " kebanyakan ", regex=True) #Convert variation of 'kebanyakan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(keberadaa)\b", " keberadaan ", regex=True) #Convert variation of 'keberadaan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kebetuln)\b", " kebetulan ", regex=True) #Convert variation of 'kebetulan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dkt|dkat|dekt|dokat|kt)\b", " dekat ", regex=True) #Convert variation of 'dekat'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tulih|tlis)\b", " tulis ", regex=True) #Convert variation of 'tulis'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bli|boli)\b", " beli ", regex=True) #Convert variation of 'beli'
tweet_text['text'] = tweet_text['text'].replace(r"\b(men|maen)\b", " main ", regex=True) #Convert variation of 'main'
tweet_text['text'] = tweet_text['text'].replace(r"\b(menan)\b", " mainan ", regex=True) #Convert variation of 'mainan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bruk)\b", " buruk ", regex=True) #Convert variation of 'buruk'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sib)\b", " nasib ", regex=True) #Convert variation of 'nasib'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tulung|tulong|tlg|tlong|tolong|tolo)\b", " tolong ", regex=True) #Convert variation of 'tolong'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bukak|buke|bukok|buko)\b", " buka ", regex=True) #Convert variation of 'buka'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mase|maso|mse)\b", " masa ", regex=True) #Convert variation of 'masa'
tweet_text['text'] = tweet_text['text'].replace(r"\b(manih)\b", " manis ", regex=True) #Convert variation of 'manis'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tgk|tgok|tngok|tngk|tengok)\b", " tengok ", regex=True) #Convert variation of 'tengok'
tweet_text['text'] = tweet_text['text'].replace(r"\b(make|mkn|makei|makang|mkang|makey|mkan)\b", " makan ", regex=True) #Convert variation of 'makan'
tweet_text['text'] = tweet_text['text'].replace(r"(sb|seb|sbab|sebp)\b", " sebab ", regex=True) #Convert variation of 'sebab'
tweet_text['text'] = tweet_text['text'].replace(r"\b(disbkn)\b", " disebabkan ", regex=True) #Convert variation of 'disebabkan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pompuan|pempuan|prmpn|prmpuan|ppuan|pmpn|mpuan)\b", " perempuan ", regex=True) #Convert variation of 'perempuan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bacakak)\b", " bertekak ", regex=True) #Convert variation of 'bertekak'
tweet_text['text'] = tweet_text['text'].replace(r"(ckp|ckap|cakak)", "cakap", regex=True) #Convert variation of 'cakap'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bacakap)\b", " bercakap ", regex=True) #Convert variation of 'bercakap'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bosku)\b", " bos aku ", regex=True) #Convert variation of 'bos aku'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bkn|bukang|bukei|bukn|bkan|bukey)\b", " bukan ", regex=True) #Convert variation of 'bukan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pikiaq|pkiaq|mikir|pk|pikir|fkir|pike|fk|fiki)\b", " fikir ", regex=True) #Convert variation of 'fikir'
tweet_text['text'] = tweet_text['text'].replace(r"\b(terpikir)\b", " terfikir ", regex=True) #Convert variation of 'terfikir'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pikiran)\b", " fikiran ", regex=True) #Convert variation of 'fikiran'
tweet_text['text'] = tweet_text['text'].replace(r"\b(wat|wak|wk|wt|buad|uat|bt|buek)\b", " buat ", regex=True) #Convert variation of 'buat'
tweet_text['text'] = tweet_text['text'].replace(r"\b(wape)\b", " buat apa ", regex=True) #Convert variation of 'buat apa'
tweet_text['text'] = tweet_text['text'].replace(r"\b(rd)\b", " third ", regex=True) #Convert variation of 'third'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kt)\b", " kat ", regex=True) #Convert variation of 'kat'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tp|tpi|tapi)\b", " tapi ", regex=True) #Convert variation of 'tapi'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dgn|dengey|dgan|dngan|dngn|ngan|ngn|nge|nga|denge|gan)\b", " dengan ", regex=True) #Convert variation of 'dengan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(rile|leklek|lek|lex|rileks|relax|rilek)\b", " relaks ", regex=True) #Convert variation of 'relaks'
tweet_text['text'] = tweet_text['text'].replace(r"\b(buleh|leh|blh|boleh|bleh|bole|bulih|boley)\b", " boleh ", regex=True) #Convert variation of 'boleh'
tweet_text['text'] = tweet_text['text'].replace(r"\b(smua|sume|semuo|semo|smeua|shemua)\b", " semua ", regex=True) #Convert variation of 'semua'
tweet_text['text'] = tweet_text['text'].replace(r"\b(td|tdi)\b", " tadi ", regex=True) #Convert variation of 'tadi'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mintak|mnta|mntk|mntak|mntok|mnx|mintk|mintok|mitok|mtk)\b", " minta ", regex=True) #Convert variation of 'minta'
tweet_text['text'] = tweet_text['text'].replace(r"\b(smlm|smlam|smalm|semalem)\b", " semalam ", regex=True) #Convert variation of 'semalam'
tweet_text['text'] = tweet_text['text'].replace(r"\b(smbil|smbel)\b", " sambil ", regex=True) #Convert variation of 'sambil'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tau|thu|taw|tawu|reti)\b", " tahu ", regex=True) #Convert variation of 'tahu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(shaja|shj)\b", " sahaja ", regex=True) #Convert variation of 'sahaja'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sj|je|jer|ja|jah|jo|jh|daje|jew|sajo|saje|joh|jek|jha|jw)\b", " saja ", regex=True) #Convert variation of 'saja'
tweet_text['text'] = tweet_text['text'].replace(r"\b(supo|sarupo|serupo|seropa)\b", " serupa ", regex=True) #Convert variation of 'serupa'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bori|buwi|bri)\b", " beri ", regex=True) #Convert variation of 'beri'
tweet_text['text'] = tweet_text['text'].replace(r"(lpa|luper|lupo|lupe|lopa|lupak)\b", " lupa ", regex=True) #Convert variation of 'lupa'
tweet_text['text'] = tweet_text['text'].replace(r"(lupoan)\b", " lupakan ", regex=True) #Convert variation of 'lupakan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(yg|yag|yng|yng|y|yg)\b", " yang ", regex=True) #Convert variation of 'yang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(utk|untk|untuk|ntuk|untuak)\b", " untuk ", regex=True) #Convert variation of 'untuk'
tweet_text['text'] = tweet_text['text'].replace(r"\b(paham|phm|fhm|pehe|pahei)", " faham\n ", regex=True) #Convert variation of 'faham'
tweet_text['text'] = tweet_text['text'].replace(r"\b(lpar|lpr|lapa|lapor|lapaq|lapo)", " lapar\n ", regex=True) #Convert variation of 'lapar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tah|nta+h)\b", " entah ", regex=True) #Convert variation of 'entah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(biase|bese|bsa)\b", " biasa ", regex=True) #Convert variation of 'biasa'
tweet_text['text'] = tweet_text['text'].replace(r"\b(rndu)\b", " rindu ", regex=True) #Convert variation of 'rindu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ptut|pt|patot|patuik)\b", " patut ", regex=True) #Convert variation of 'patut'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kodukut|kedek|kede|kedeq)\b", " kedekut ", regex=True) #Convert variation of 'kedekut'
tweet_text['text'] = tweet_text['text'].replace(r"\b(srabut|srbut)\b", " serabut ", regex=True) #Convert variation of 'serabut'
tweet_text['text'] = tweet_text['text'].replace(r"\b(rmbut|rmbt|qambut)\b", " rambut ", regex=True) #Convert variation of 'rambut'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mst|msti|msty)\b", " mesti ", regex=True) #Convert variation of 'mesti'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mestilah)\b", " mesti la ", regex=True) #Convert variation of 'mesti la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(beno|benor|bebenor)\b", " benar ", regex=True) #Convert variation of 'benar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kbo|kobar|kabo|kabor|kaba|kabar|koba)\b", " khabar ", regex=True) #Convert variation of 'khabar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sbg|sbgai)\b", " sebagai ", regex=True) #Convert variation of 'sebagai'
tweet_text['text'] = tweet_text['text'].replace(r"\b(balek|blik|blk|alik)\b", " balik ", regex=True) #Convert variation of 'balik'
tweet_text['text'] = tweet_text['text'].replace(r"\b(brblik)\b", " berbalik ", regex=True) #Convert variation of 'berbalik'
tweet_text['text'] = tweet_text['text'].replace(r"\b(beselerak)\b", " berselerak ", regex=True) #Convert variation of 'berselerak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(caye|prcaye|prcaya|prcye|prcya|caya|percaya)", " percaya ", regex=True) #Convert variation of 'percaya'
tweet_text['text'] = tweet_text['text'].replace(r"\b(seyes|srs)\b", " serius ", regex=True) #Convert variation of 'serius'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kls|klas|kelaih)\b", " kelas ", regex=True) #Convert variation of 'kelas'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ngis|nagis)\b", " nangis ", regex=True) #Convert variation of 'nangis'
tweet_text['text'] = tweet_text['text'].replace(r"(raso|rse|rsa|rase|rso|qasa)\b", " rasa ", regex=True) #Convert variation of 'rasa'
tweet_text['text'] = tweet_text['text'].replace(r"\b(beletiaq)\b", " berleter ", regex=True) #Convert variation of 'berleter'
tweet_text['text'] = tweet_text['text'].replace(r"\b(berenti|benti|brhnti|berhenti|beti|bhenti)\b", " berhenti ", regex=True) #Convert variation of 'berhenti'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sbenar|sbnq|sbana|sbenar|sbna|sbnr|sebenaq)\b", " sebenar ", regex=True) #Convert variation of 'sebenar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bodo|buduh|bedooh)\b", " bodoh ", regex=True) #Convert variation of 'bodoh'
tweet_text['text'] = tweet_text['text'].replace(r"\b(jgn|jange|jangei|jngan|jngn)", " jangan\n ", regex=True) #Convert variation of 'jangan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tdk|tdak|tidk|tidak|ndak|idok|nggak|indak|saindak)\b", " tidak ", regex=True) #Convert variation of 'tidak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(hanat|hanet)\b", " lahanat ", regex=True) #Convert variation of 'lahanat'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bgst|bngsat)\b", " bangsat ", regex=True) #Convert variation of 'bangsat'
tweet_text['text'] = tweet_text['text'].replace(r"\b(cb)\b", " cibai ", regex=True) #Convert variation of 'cibai'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kengkadang|kekadang|kenkadang|knkdng|kgkdang|kgkdng|kdg|kdang)\b", " kadang ", regex=True) #Convert variation of 'kadang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(peparu|paru)\b", " paru ", regex=True) #Convert variation of 'paruparu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(budok|bdk|bebudak)\b", " budak ", regex=True) #Convert variation of 'budak'
tweet_text['text'] = tweet_text['text'].replace(r"(jalang|jejln|jenjalan|jnjln|jenjln|jenjlan|jln|jlan|jale|jaln|sajalan)\b", "jalan ", regex=True) #Convert variation of 'jalan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(lelebih|lbih|lebey|lebe|lelenih)\b", " lebih ", regex=True) #Convert variation of 'lebih'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tiba|tibo|tetiba|tibe|tiber|tetibe)\b", " tiba ", regex=True) #Convert variation of 'tiba'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kengkonon|kenkonon|knknon)\b", " konon ", regex=True) #Convert variation of 'konon'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sklh|skolah|skolh|skoloh|skolo|skola|sekolah|sekoloh|kola|kolah)", " sekolah\n ", regex=True) #Convert variation of 'sekolah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(masaloh|msalah|mslh|mslah|msla)\b", " masalah ", regex=True) #Convert variation of 'masalah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(adala|adela|adelah)\b", " ada la ", regex=True) #Convert variation of 'ada la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(slah|slh|saloh|salah|sloh)", " salah ", regex=True) #Convert variation of 'salah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(salahkah)", " salah ke ", regex=True) #Convert variation of 'salah ke'
tweet_text['text'] = tweet_text['text'].replace(r"\b(slhkan)", " salahkan ", regex=True) #Convert variation of 'salahkan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bersaloh|besalah)\b", " bersalah ", regex=True) #Convert variation of 'bersalah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sblah|seblah|sebelaj|sebeloh|sebelh|sbelah)\b", " sebelah ", regex=True) #Convert variation of 'sebelah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sblm|sbelom|sebelom|sblom|sblum)\b", " sebelum ", regex=True) #Convert variation of 'sebelum'
tweet_text['text'] = tweet_text['text'].replace(r"\b(klah|kaloh)\b", " kalah ", regex=True) #Convert variation of 'kalah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ade|ad|ado|de|dk|dei|dek|hado)\b", " ada", regex=True) #Convert variation of 'ada'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kpd|kepade|kpada|kpade|kepado)\b", " kepada ", regex=True) #Convert variation of 'kepada'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pala|palo|paler|pale|kpla|kpala|kpale|kepale|kapalo)\b", " kepala ", regex=True) #Convert variation of 'kepala'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bile|bilo|bl)\b", " bila ", regex=True) #Convert variation of 'bila'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pulak|pulek|pulo|plak|plk|pulok|lak(?<!i)|lok)\b", " pula ", regex=True) #Convert variation of 'pula'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sgala|segla)\b", " segala ", regex=True) #Convert variation of 'segala'
tweet_text['text'] = tweet_text['text'].replace(r"(gile|gilo|giler|gilak|gler)\b", " gila ", regex=True) #Convert variation of 'gila'
#----------------------------------------------------------------------------------------------------------------------
tweet_text['text'] = tweet_text['text'].replace(r"\b(huduh|hudoh|uduh)\b", " hodoh ", regex=True) #Convert variation of 'hodoh'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ngail)\b", " mengail ", regex=True) #Convert variation of 'mengail'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kecik|kacik)\b", " kecil ", regex=True) #Convert variation of 'kecil'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mok|omak)\b", " mak ", regex=True) #Convert variation of 'mak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(makabah)\b", " mak abah ", regex=True) #Convert variation of 'mak abah'
tweet_text['text'] = tweet_text['text'].replace(r"(ketor|ketaq)\b", " ketar ", regex=True) #Convert variation of 'ketar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sribu)\b", " seribu ", regex=True) #Convert variation of 'seribu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bapak|bapok)\b", " bapa ", regex=True) #Convert variation of 'bapa'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bia|biaq|bior)\b", " biar ", regex=True) #Convert variation of 'biar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(cuko|cukor)\b", " cukur ", regex=True) #Convert variation of 'cukur'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kesian|sian|cian)\b", " kasihan ", regex=True) #Convert variation of 'kasihan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ksiankan)\b", " kasihankan ", regex=True) #Convert variation of 'kasihankan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ma)\b", " mama ", regex=True) #Convert variation of 'mama'
tweet_text['text'] = tweet_text['text'].replace(r"\b(beci|bencik|menci)\b", " benci ", regex=True) #Convert variation of 'benci'
tweet_text['text'] = tweet_text['text'].replace(r"\b(buboh|bui|buh)\b", " bubuh ", regex=True) #Convert variation of 'bubuh'
tweet_text['text'] = tweet_text['text'].replace(r"\b(do+h)\b", " doh ", regex=True) #Convert variation of 'doh'
tweet_text['text'] = tweet_text['text'].replace(r"\b(gemok)\b", " gemuk ", regex=True) #Convert variation of 'gemuk'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ar|loe|loh|lha|lh|la+h)\b", " la ", regex=True) #Convert variation of 'la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(jare|jrg)\b", " jarang ", regex=True) #Convert variation of 'jarang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(muke|muko)\b", " muka ", regex=True) #Convert variation of 'muka'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ek|ae|eah)\b", " ea ", regex=True) #Convert variation of 'ea'
tweet_text['text'] = tweet_text['text'].replace(r"\b(keta|kete|keto)\b", " kereta ", regex=True) #Convert variation of 'kereta'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mugo)\b", " moga ", regex=True) #Convert variation of 'moga'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ngaku)\b", " mengaku ", regex=True) #Convert variation of 'mengaku'
tweet_text['text'] = tweet_text['text'].replace(r"\b(putuih|putuh)\b", " putus ", regex=True) #Convert variation of 'putus'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sutong)\b", " sotong ", regex=True) #Convert variation of 'sotong'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tajatuh)\b", " terjatuh ", regex=True) #Convert variation of 'terjatuh'
tweet_text['text'] = tweet_text['text'].replace(r"\b(caro)\b", " cara ", regex=True) #Convert variation of 'cara'
tweet_text['text'] = tweet_text['text'].replace(r"\b(deraih|derah)\b", " deras ", regex=True) #Convert variation of 'deras'
tweet_text['text'] = tweet_text['text'].replace(r"\b(duo|duak)\b", " dua ", regex=True) #Convert variation of 'dua'
tweet_text['text'] = tweet_text['text'].replace(r"\b(duduak|dodua)\b", " dua dua ", regex=True) #Convert variation of 'dua dua'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kiro|kire)\b", " kira ", regex=True) #Convert variation of 'kira'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kirala)\b", " kira la ", regex=True) #Convert variation of 'kira la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(skira)\b", " sekira ", regex=True) #Convert variation of 'sekira'
tweet_text['text'] = tweet_text['text'].replace(r"\b(nate|natey|ntg)\b", " natang ", regex=True) #Convert variation of 'natang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(natepo)\b", " natang apa ", regex=True) #Convert variation of 'natang apa'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ngantuk|mengntuk|ngatok)\b", " mengantuk ", regex=True) #Convert variation of 'mengantuk'
tweet_text['text'] = tweet_text['text'].replace(r"\b(peh|perg+h+)\b", " pergh ", regex=True) #Convert variation of 'pergh'
tweet_text['text'] = tweet_text['text'].replace(r"\b(perati|peghati)\b", " memerhati ", regex=True) #Convert variation of 'memerhati'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tanoh)\b", " tanah ", regex=True) #Convert variation of 'tanah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tauk)\b", " taruk ", regex=True) #Convert variation of 'taruk'
tweet_text['text'] = tweet_text['text'].replace(r"\b(akai)\b", " akal ", regex=True) #Convert variation of 'akal'
tweet_text['text'] = tweet_text['text'].replace(r"\b(basa)\b", " basah ", regex=True) #Convert variation of 'basah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(baso|basuah)\b", " basuh ", regex=True) #Convert variation of 'basuh'
tweet_text['text'] = tweet_text['text'].replace(r"\b(jumpe|jumpo|jupo|jmpa|jmpe|jmpo|jupe|jumpak)\b", " jumpa ", regex=True) #Convert variation of 'jumpa'
tweet_text['text'] = tweet_text['text'].replace(r"\b(maningga|meninggai)\b", " meninggal ", regex=True) #Convert variation of 'meninggal'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ngaji)\b", " mengaji ", regex=True) #Convert variation of 'mengaji'
tweet_text['text'] = tweet_text['text'].replace(r"\b(nikoh)\b", " nikah ", regex=True) #Convert variation of 'nikah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(noh|nh)\b", " nah ", regex=True) #Convert variation of 'nah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(puah|puaih)\b", " puas ", regex=True) #Convert variation of 'puas'
tweet_text['text'] = tweet_text['text'].replace(r"\b(paiti)", " sampai hati ", regex=True) #Convert variation of 'sampai hati'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sampailah)", " sampai la ", regex=True) #Convert variation of 'sampai la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(slama)", " selama ", regex=True) #Convert variation of 'selama'
tweet_text['text'] = tweet_text['text'].replace(r"\b(slmani)", " selama ini ", regex=True) #Convert variation of 'selama ini'
tweet_text['text'] = tweet_text['text'].replace(r"\b(aloh|alah|lahai)\b", " alahai ", regex=True) #Convert variation of 'alahai'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ati|aty)\b", " hati ", regex=True) #Convert variation of 'hati'
tweet_text['text'] = tweet_text['text'].replace(r"\b(hatiku)\b", " hati aku ", regex=True) #Convert variation of 'hati aku'
tweet_text['text'] = tweet_text['text'].replace(r"\b(banggo|bangge)\b", " bangga ", regex=True) #Convert variation of 'bangga'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bantai)\b", " bantal ", regex=True) #Convert variation of 'bantal'
tweet_text['text'] = tweet_text['text'].replace(r"\b(beraih)\b", " beras ", regex=True) #Convert variation of 'beras'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bialah|biala)\b", " biar la ", regex=True) #Convert variation of 'biar la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bio)\b", " biologi ", regex=True) #Convert variation of 'biologi'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ilang|hile)\b", " hilang ", regex=True) #Convert variation of 'hilang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(jate|jntan|jntn)\b", " jantan ", regex=True) #Convert variation of 'jantan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(jubo)\b", " jubur ", regex=True) #Convert variation of 'jubur'
tweet_text['text'] = tweet_text['text'].replace(r"\b(lmbt|lmbat)\b", " lambat ", regex=True) #Convert variation of 'lambat'
tweet_text['text'] = tweet_text['text'].replace(r"\b(luaq|luor|luo)\b", " luar ", regex=True) #Convert variation of 'luar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mato|mater)\b", " mata ", regex=True) #Convert variation of 'mata'
tweet_text['text'] = tweet_text['text'].replace(r"\b(muroh)\b", " murah ", regex=True) #Convert variation of 'murah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(nasik)\b", " nasi ", regex=True) #Convert variation of 'nasi'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ngoh)\b", " ngah ", regex=True) #Convert variation of 'ngah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(puok)\b", " puak ", regex=True) #Convert variation of 'puak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(rayo)\b", " raya ", regex=True) #Convert variation of 'raya'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sedara)\b", " saudara ", regex=True) #Convert variation of 'saudara'
tweet_text['text'] = tweet_text['text'].replace(r"\b(semaye|smyg|sumbayang|sumbahyang|smaye|semayang|maye)\b", " sembahyang ", regex=True) #Convert variation of 'sembahyang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(senin)\b", " isnin ", regex=True) #Convert variation of 'isnin'
tweet_text['text'] = tweet_text['text'].replace(r"\b(spita)\b", " hospital ", regex=True) #Convert variation of 'hospital'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sutan)\b", " sultan ", regex=True) #Convert variation of 'sultan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tawe)\b", " tawa ", regex=True) #Convert variation of 'tawa'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tgn)\b", " tangan ", regex=True) #Convert variation of 'tangan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tobat)\b", " taubat ", regex=True) #Convert variation of 'taubat'
tweet_text['text'] = tweet_text['text'].replace(r"\b(yalah|yelah|yup|yela|iyolah)\b", " ya la ", regex=True) #Convert variation of 'ya la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(aboh)\b", " abah ", regex=True) #Convert variation of 'abah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ajet)\b", " bajet ", regex=True) #Convert variation of 'bajet'
tweet_text['text'] = tweet_text['text'].replace(r"\b(baduo)\b", " berdua ", regex=True) #Convert variation of 'berdua'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bangla)\b", " bangladesh ", regex=True) #Convert variation of 'bangladesh'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bangon|bagun)\b", " bangun ", regex=True) #Convert variation of 'bangun'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bantuak)\b", " bantu ", regex=True) #Convert variation of 'bantu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(barampek)\b", " berempat ", regex=True) #Convert variation of 'berempat'
tweet_text['text'] = tweet_text['text'].replace(r"\b(basatu)\b", " bersatu ", regex=True) #Convert variation of 'bersatu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bataduah)\b", " berteduh ", regex=True) #Convert variation of 'berteduh'
tweet_text['text'] = tweet_text['text'].replace(r"\b(belambak)\b", " berlambak ", regex=True) #Convert variation of 'berlambak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(beranok)\b", " beranak ", regex=True) #Convert variation of 'beranak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bereh)\b", " beres ", regex=True) #Convert variation of 'beres'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bgus|bgs|ba+gu+h|ba+gu+i+h+)\b", " bagus ", regex=True) #Convert variation of 'bagus'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bontot)\b", " buntut ", regex=True) #Convert variation of 'buntut'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bsuara)\b", " bersuara ", regex=True) #Convert variation of 'bersuara'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bungo)\b", " bunga ", regex=True) #Convert variation of 'bunga'
tweet_text['text'] = tweet_text['text'].replace(r"\b(celoh)\b", " celah ", regex=True) #Convert variation of 'celah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ceq)\b", " cek ", regex=True) #Convert variation of 'cek'
tweet_text['text'] = tweet_text['text'].replace(r"\b(cinto)\b", " cinta ", regex=True) #Convert variation of 'cinta'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bercintalah)\b", " bercinta la ", regex=True) #Convert variation of 'bercinta la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(debor)\b", " debar ", regex=True) #Convert variation of 'debar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(deki)\b", " dengki ", regex=True) #Convert variation of 'dengki'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dijua)\b", " di jual ", regex=True) #Convert variation of 'dijual'
tweet_text['text'] = tweet_text['text'].replace(r"\b(doso)\b", " dosa ", regex=True) #Convert variation of 'dosa'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dtgla)\b", " datang la ", regex=True) #Convert variation of 'datang la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dunio)\b", " dunia ", regex=True) #Convert variation of 'dunia'
tweet_text['text'] = tweet_text['text'].replace(r"\b(galih|galeh|kalih|kaleh)\b", " beralih ", regex=True) #Convert variation of 'beralih'
tweet_text['text'] = tweet_text['text'].replace(r"\b(gatai)\b", " gatal ", regex=True) #Convert variation of 'gatal'
tweet_text['text'] = tweet_text['text'].replace(r"\b(gegor|ggr)\b", " gegar ", regex=True) #Convert variation of 'gegar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(gerok)\b", " gerak ", regex=True) #Convert variation of 'gerak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(geraklu|raklu|graklu)\b", " gerak dulu ", regex=True) #Convert variation of 'gerak dulu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(gue|gwa|gw|wa)\b", " gua ", regex=True) #Convert variation of 'gua'
tweet_text['text'] = tweet_text['text'].replace(r"\b(helok|leklok)\b", " elok ", regex=True) #Convert variation of 'elok'
tweet_text['text'] = tweet_text['text'].replace(r"\b(karan)\b", " elektrik ", regex=True) #Convert variation of 'elektrik'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kerane|karena|krna|krne|krn|kerna)\b", " kerana ", regex=True) #Convert variation of 'kerana'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kelupoq|kelupur)\b", " gelupur ", regex=True) #Convert variation of 'gelupur'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kemana|kemane|kmane)\b", " ke mana ", regex=True) #Convert variation of 'ke mana'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kesah|kesoh|kesa|isah)\b", " kisah ", regex=True) #Convert variation of 'kisah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ksih)\b", " kasih ", regex=True) #Convert variation of 'kasih'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kuruih)\b", " kurus ", regex=True) #Convert variation of 'kurus'
tweet_text['text'] = tweet_text['text'].replace(r"\b(lajok)\b", " lajak ", regex=True) #Convert variation of 'lajak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(lamak)\b", " lemak ", regex=True) #Convert variation of 'lemak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(langgo|langgor)\b", " langgar ", regex=True) #Convert variation of 'langgar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(lawo)\b", " lawa ", regex=True) #Convert variation of 'lawa'
tweet_text['text'] = tweet_text['text'].replace(r"\b(lemoh)\b", " lemah ", regex=True) #Convert variation of 'lemah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(lgsg|langsuang|lasung|lgsung)\b", " langsung ", regex=True) #Convert variation of 'langsung'
tweet_text['text'] = tweet_text['text'].replace(r"\b(lgu)\b", " lagu ", regex=True) #Convert variation of 'lagu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(maap)\b", " maaf ", regex=True) #Convert variation of 'maaf'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mahai)\b", " mahal ", regex=True) #Convert variation of 'mahal'
tweet_text['text'] = tweet_text['text'].replace(r"\b(makane)\b", " makanan ", regex=True) #Convert variation of 'makanan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(malamar)\b", " melamar ", regex=True) #Convert variation of 'melamar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mamdarin)\b", " mandarin ", regex=True) #Convert variation of 'mandarin'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mancuci)\b", " mencuci ", regex=True) #Convert variation of 'mencuci'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mandeh)\b", " mandi ", regex=True) #Convert variation of 'mandi'
tweet_text['text'] = tweet_text['text'].replace(r"\b(menyesai|mnyesal|nyesal)\b", " menyesal ", regex=True) #Convert variation of 'menyesal'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mgkin|mngkin)\b", " mungkin ", regex=True) #Convert variation of 'mungkin'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mkcik)\b", " makcik ", regex=True) #Convert variation of 'makcik'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mksud)\b", " maksud ", regex=True) #Convert variation of 'maksud'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mpi)\b", " mimpi ", regex=True) #Convert variation of 'mimpi'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ngisap)\b", " menghisap ", regex=True) #Convert variation of 'menghisap'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ngukit)\b", " mengungkit ", regex=True) #Convert variation of 'mengungkit'
tweet_text['text'] = tweet_text['text'].replace(r"\b(nyumbang)\b", " menyumbang ", regex=True) #Convert variation of 'menyumbang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pata)\b", " pantai ", regex=True) #Convert variation of 'pantai'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pdh)\b", " pindah ", regex=True) #Convert variation of 'pindah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(penang)\b", " pulau pinang ", regex=True) #Convert variation of 'pulau pinang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(perbetui)\b", " perbetulkan ", regex=True) #Convert variation of 'perbetulkan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pj)\b", " putrajaya ", regex=True) #Convert variation of 'putrajaya'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pndangan)\b", " pandangan ", regex=True) #Convert variation of 'pandangan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(prasaan|persaan)\b", " perasaan ", regex=True) #Convert variation of 'perasaan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ptg)\b", " petang ", regex=True) #Convert variation of 'petang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sadang|sdg|sdang)\b", " sedang ", regex=True) #Convert variation of 'sedang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(samalah|samala)\b", " sama la ", regex=True) #Convert variation of 'sama la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sedo|sadar|seda|sedor)\b", " sedar ", regex=True) #Convert variation of 'sedar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sekor)\b", " seekor ", regex=True) #Convert variation of 'seekor'
tweet_text['text'] = tweet_text['text'].replace(r"\b(shalat)\b", " solat ", regex=True) #Convert variation of 'solat'
tweet_text['text'] = tweet_text['text'].replace(r"\b(slang)\b", " slanga ", regex=True) #Convert variation of 'slanga'
tweet_text['text'] = tweet_text['text'].replace(r"\b(smbung)\b", " sambung ", regex=True) #Convert variation of 'sambung'
tweet_text['text'] = tweet_text['text'].replace(r"\b(smpah)\b", " sampah ", regex=True) #Convert variation of 'sampah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(smpaikan)\b", " sampai kan ", regex=True) #Convert variation of 'sampai kan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sorga)\b", " syurga ", regex=True) #Convert variation of 'syurga'
tweet_text['text'] = tweet_text['text'].replace(r"\b(supayo)\b", " supaya ", regex=True) #Convert variation of 'supaya'
tweet_text['text'] = tweet_text['text'].replace(r"\b(talingo)\b", " telinga ", regex=True) #Convert variation of 'telinga'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tino|betine)\b", " betina ", regex=True) #Convert variation of 'betina'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tiwala)\b", " tuala ", regex=True) #Convert variation of 'tuala'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tolongla|tolonglah)\b", " tolong la ", regex=True) #Convert variation of 'tolong la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ujung)\b", " hujung ", regex=True) #Convert variation of 'hujung'
tweet_text['text'] = tweet_text['text'].replace(r"\b(untuang)\b", " untung ", regex=True) #Convert variation of 'untung'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ustat)\b", " ustaz ", regex=True) #Convert variation of 'ustaz'
tweet_text['text'] = tweet_text['text'].replace(r"\b(utagha)\b", " utara ", regex=True) #Convert variation of 'utara'
tweet_text['text'] = tweet_text['text'].replace(r"\b(vid|videos)\b", " video ", regex=True) #Convert variation of 'video'
tweet_text['text'] = tweet_text['text'].replace(r"\b(wktu)\b", " waktu ", regex=True) #Convert variation of 'waktu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(abuk)\b", " habuk ", regex=True) #Convert variation of 'habuk'
tweet_text['text'] = tweet_text['text'].replace(r"\b(adaja)\b", " ada saja ", regex=True) #Convert variation of 'ada saja'
tweet_text['text'] = tweet_text['text'].replace(r"\b(adiku)\b", " adik ku ", regex=True) #Convert variation of 'adik ku'
tweet_text['text'] = tweet_text['text'].replace(r"\b(agkt|akat)\b", " angkat ", regex=True) #Convert variation of 'angkat'
tweet_text['text'] = tweet_text['text'].replace(r"\b(amboih|amboi+|hamboih)\b", " amboi ", regex=True) #Convert variation of 'amboi'
tweet_text['text'] = tweet_text['text'].replace(r"\b(asik)\b", " asyik ", regex=True) #Convert variation of 'asyik'
tweet_text['text'] = tweet_text['text'].replace(r"\b(atuq)\b", " datuk ", regex=True) #Convert variation of 'datuk'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ayt)\b", " ayat ", regex=True) #Convert variation of 'ayat'
tweet_text['text'] = tweet_text['text'].replace(r"\b(badoa)\b", " berdoa ", regex=True) #Convert variation of 'berdoa'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bagak)\b", " berlagak ", regex=True) #Convert variation of 'berlagak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bagindo)\b", " baginda ", regex=True) #Convert variation of 'baginda'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bahagio|bahgia)\b", " bahagia ", regex=True) #Convert variation of 'bahagia'
tweet_text['text'] = tweet_text['text'].replace(r"\b(baharago)\b", " berharga ", regex=True) #Convert variation of 'berharga'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bahayo|bahya)\b", " bahaya ", regex=True) #Convert variation of 'bahaya'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bahawo|bahwa)\b", " bahawa ", regex=True) #Convert variation of 'bahawa'
tweet_text['text'] = tweet_text['text'].replace(r"\b(halahmak)\b", " alamak ", regex=True) #Convert variation of 'alamak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bdn)\b", " badan ", regex=True) #Convert variation of 'badan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bgai)\b", " bagai ", regex=True) #Convert variation of 'bagai'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bhang)\b", " bahang ", regex=True) #Convert variation of 'bahang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(baq|bk)\b", " bak ", regex=True) #Convert variation of 'bak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(banda|bando|bandor)\b", " bandar ", regex=True) #Convert variation of 'bandar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bangse|bngse|bngsa)\b", " bangsa ", regex=True) #Convert variation of 'bangsa'
tweet_text['text'] = tweet_text['text'].replace(r"\b(blanja|blnja|lanje|lanjio)\b", " belanja ", regex=True) #Convert variation of 'belanja'
tweet_text['text'] = tweet_text['text'].replace(r"\b(cilaka|jilake)\b", " celaka ", regex=True) #Convert variation of 'celaka'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ceqah|cera|ceroh)\b", " cerah ", regex=True) #Convert variation of 'cerah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(gembire|gumbira|burgermbira)\b", " gembira ", regex=True) #Convert variation of 'gembira'
tweet_text['text'] = tweet_text['text'].replace(r"\b(gere)\b", " geram ", regex=True) #Convert variation of 'geram'
tweet_text['text'] = tweet_text['text'].replace(r"\b(hino|hine)\b", " hina ", regex=True) #Convert variation of 'hina'
tweet_text['text'] = tweet_text['text'].replace(r"\b(hite|itam)\b", " hitam ", regex=True) #Convert variation of 'hitam'
tweet_text['text'] = tweet_text['text'].replace(r"\b(jago)\b", " jaga ", regex=True) #Convert variation of 'jaga'
tweet_text['text'] = tweet_text['text'].replace(r"\b(jagalah|jagala|jagalh|jgalah|jgala|jgalh)\b", " jaga la ", regex=True) #Convert variation of 'jaga la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(o+h+|owh)\b", " oh ", regex=True) #Convert variation of 'oh'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tkjut|tekejut|tkejut)\b", " terkejut ", regex=True) #Convert variation of 'terkejut'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kadai|keda|kdai)\b", " kedai ", regex=True) #Convert variation of 'kedai'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mabok)\b", " mabuk ", regex=True) #Convert variation of 'mabuk'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bareng)\b", " baring ", regex=True) #Convert variation of 'baring'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bareh|beraih)\b", " beras ", regex=True) #Convert variation of 'beras'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ber)\b", " baru ", regex=True) #Convert variation of 'baru'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bguna)", " berguna ", regex=True) #Convert variation of 'berguna'
tweet_text['text'] = tweet_text['text'].replace(r"\b(btg)\b", " batang ", regex=True) #Convert variation of 'batang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(banyam)\b", " bayam ", regex=True) #Convert variation of 'bayam'
tweet_text['text'] = tweet_text['text'].replace(r"\b(beloh)\b", " belah ", regex=True) #Convert variation of 'belah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bajanji)\b", " berjanji ", regex=True) #Convert variation of 'berjanji'
tweet_text['text'] = tweet_text['text'].replace(r"\b(benaq)\b", " benak ", regex=True) #Convert variation of 'benak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bakasiah)\b", " berkasih ", regex=True) #Convert variation of 'berkasih'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bntuk)\b", " bentuk ", regex=True) #Convert variation of 'bentuk'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bragas)\b", " beragas ", regex=True) #Convert variation of 'beragas'
tweet_text['text'] = tweet_text['text'].replace(r"\b(baraja)\b", " beraja ", regex=True) #Convert variation of 'beraja'
tweet_text['text'] = tweet_text['text'].replace(r"\b(beratuih)\b", " beratus ", regex=True) #Convert variation of 'beratus'
tweet_text['text'] = tweet_text['text'].replace(r"\b(brangan)\b", " berangan ", regex=True) #Convert variation of 'berangan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bakawan)\b", " berkawan ", regex=True) #Convert variation of 'berkawan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bkenan)\b", " berkenaan ", regex=True) #Convert variation of 'berkenaan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(berlado)\b", " berlada ", regex=True) #Convert variation of 'berlada'
tweet_text['text'] = tweet_text['text'].replace(r"\b(berlkon|blakon)\b", " berlakon ", regex=True) #Convert variation of 'berlakon'
tweet_text['text'] = tweet_text['text'].replace(r"\b(berlapa)\b", " berlapar ", regex=True) #Convert variation of 'berlapar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bpunya)\b", " berpunya ", regex=True) #Convert variation of 'berpunya'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bersabaq)\b", " bersabar ", regex=True) #Convert variation of 'bersabar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bersamaku)\b", " bersama aku ", regex=True) #Convert variation of 'bersama aku'
tweet_text['text'] = tweet_text['text'].replace(r"\b(basayang)\b", " bersayang ", regex=True) #Convert variation of 'bersayang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bersembangla)\b", " bersembang la ", regex=True) #Convert variation of 'bersembang la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(besungguh)\b", " bersungguh ", regex=True) #Convert variation of 'bersungguh'
tweet_text['text'] = tweet_text['text'].replace(r"\b(berthn)\b", " bertahun ", regex=True) #Convert variation of 'bertahun'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bertmbh)\b", " bertambah ", regex=True) #Convert variation of 'bertambah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(batanyo)\b", " bertanya ", regex=True) #Convert variation of 'bertanya'
tweet_text['text'] = tweet_text['text'].replace(r"\b(berterabuq)\b", " berterabur ", regex=True) #Convert variation of 'berterabur'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bteriak)\b", " berteriak ", regex=True) #Convert variation of 'berteriak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bertdung)\b", " bertudung ", regex=True) #Convert variation of 'bertudung'
tweet_text['text'] = tweet_text['text'].replace(r"\b(btunang)\b", " bertunang ", regex=True) #Convert variation of 'bertunang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(beruboh)\b", " berubah ", regex=True) #Convert variation of 'berubah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(berok)\b", " beruk ", regex=True) #Convert variation of 'beruk'
tweet_text['text'] = tweet_text['text'].replace(r"\b(berule)\b", " berulang ", regex=True) #Convert variation of 'berulang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(betape)\b", " betapa ", regex=True) #Convert variation of 'betapa'
tweet_text['text'] = tweet_text['text'].replace(r"\b(betula|betullah)\b", " betul la ", regex=True) #Convert variation of 'betul la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bgt)\b", " bangat ", regex=True) #Convert variation of 'bangat'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bede)\b", " beza ", regex=True) #Convert variation of 'beza'
tweet_text['text'] = tweet_text['text'].replace(r"\b(biasalah)\b", " biasa la ", regex=True) #Convert variation of 'biasa la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bijok)\b", " bijak ", regex=True) #Convert variation of 'bijak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bncang)\b", " bincang ", regex=True) #Convert variation of 'bincang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bodola)\b", " bodoh la ", regex=True) #Convert variation of 'bodoh la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bulihko)\b", " boleh ke ", regex=True) #Convert variation of 'boleh ke'
tweet_text['text'] = tweet_text['text'].replace(r"\b(buoh)\b", " buah ", regex=True) #Convert variation of 'buah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(buje)\b", " bujang ", regex=True) #Convert variation of 'bujang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bukik)\b", " bukit ", regex=True) #Convert variation of 'bukit'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bukitinggi)\b", " bukit tinggi ", regex=True) #Convert variation of 'bukit tinggi'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bgkus)\b", " bungkus ", regex=True) #Convert variation of 'bungkus'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bunyik)\b", " bunyi ", regex=True) #Convert variation of 'bunyi'
tweet_text['text'] = tweet_text['text'].replace(r"\b(cdang)\b", " cadang ", regex=True) #Convert variation of 'cadang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(cdangkn|cdangkan)\b", " cadangkan ", regex=True) #Convert variation of 'cadangkan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(capur)\b", " campur ", regex=True) #Convert variation of 'campur'
tweet_text['text'] = tweet_text['text'].replace(r"\b(cou)\b", " cau ", regex=True) #Convert variation of 'cau'
tweet_text['text'] = tweet_text['text'].replace(r"\b(cicoh)\b", " cecah ", regex=True) #Convert variation of 'cecah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(cintamu)\b", " cinta kamu ", regex=True) #Convert variation of 'cinta kamu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(cekelat)\b", " coklat ", regex=True) #Convert variation of 'coklat'
tweet_text['text'] = tweet_text['text'].replace(r"\b(cungak)\b", " congak ", regex=True) #Convert variation of 'congak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dado)\b", " dada ", regex=True) #Convert variation of 'dada'
tweet_text['text'] = tweet_text['text'].replace(r"\b(knak|knk)\b", " kanak ", regex=True) #Convert variation of 'kanak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(koto|kotok)\b", " kotor ", regex=True) #Convert variation of 'kotor'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kuang|kure|kureng)\b", " kurang ", regex=True) #Convert variation of 'kurang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kuajaq)\b", " kurang ajar ", regex=True) #Convert variation of 'kurang ajar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(makno|mkne)\b", " makna ", regex=True) #Convert variation of 'makna'
tweet_text['text'] = tweet_text['text'].replace(r"\b(manje|manje+w|memanja)\b", " manja ", regex=True) #Convert variation of 'manja'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mngamuk|ngamok)\b", " mengamuk ", regex=True) #Convert variation of 'mengamuk'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mngata|ngate)\b", " mengata ", regex=True) #Convert variation of 'mengata'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mdh|mudoh)\b", " mudah ", regex=True) #Convert variation of 'mudah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pakwa|pkwe|pokwe)\b", " pakwe ", regex=True) #Convert variation of 'pakwe'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mkwe)\b", " makwe ", regex=True) #Convert variation of 'makwe'
tweet_text['text'] = tweet_text['text'].replace(r"\b(panaih|pns)\b", " panas ", regex=True) #Convert variation of 'panas'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pndang|pndg)\b", " pandang ", regex=True) #Convert variation of 'pandang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pedah|pedaih|podeh)\b", " pedas ", regex=True) #Convert variation of 'pedas'
tweet_text['text'] = tweet_text['text'].replace(r"\b(petama|petame)\b", " pertama ", regex=True) #Convert variation of 'pertama'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pisanh|pisng)\b", " pisang ", regex=True) #Convert variation of 'pisang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(giso|riso|iso)\b", " risau ", regex=True) #Convert variation of 'risau'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sdak)\b", " sedak ", regex=True) #Convert variation of 'sedak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(scara|scare)\b", " secara ", regex=True) #Convert variation of 'secara'
tweet_text['text'] = tweet_text['text'].replace(r"\b(nunjuk|tunjok)\b", " tunjuk ", regex=True) #Convert variation of 'tunjuk'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dmnsara)\b", " damansara ", regex=True) #Convert variation of damansara'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dendiam)\b", " diam ", regex=True) #Convert variation of diam'
tweet_text['text'] = tweet_text['text'].replace(r"\b(diateh)\b", " di atas ", regex=True) #Convert variation of diatas'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dibuek)\b", " di buat ", regex=True) #Convert variation of dibuat'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dibetolkan)\b", " dibetulkan ", regex=True) #Convert variation of dibetulkan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dayet)\b", " diet ", regex=True) #Convert variation of diet'
tweet_text['text'] = tweet_text['text'].replace(r"\b(digantuang)\b", " di gantung ", regex=True) #Convert variation of di gantung'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dhdpan)\b", " di hadapan ", regex=True) #Convert variation of di hadapan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dijamua)\b", " di jamu ", regex=True) #Convert variation of di jamu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dikumpulin)\b", " dikumpulkan ", regex=True) #Convert variation of dikumpulkan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dimane|dimana)\b", " di mana ", regex=True) #Convert variation of di mana'
tweet_text['text'] = tweet_text['text'].replace(r"\b(deeperbodohkan)\b", " diperbodohkan ", regex=True) #Convert variation of diperbodohkan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(deepermainkan)\b", " dipermainkan ", regex=True) #Convert variation of dipermainkan
tweet_text['text'] = tweet_text['text'].replace(r"\b(deepersalahkan)\b", " dipersalahkan ", regex=True) #Convert variation of dipersalahkan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dirik)\b", " diri ", regex=True) #Convert variation of diri'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dirimu)\b", " diri kamu ", regex=True) #Convert variation of diri kamu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dirumah|drumah)\b", " di rumah ", regex=True) #Convert variation of di rumah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(disbkan)\b", " disebabkan ", regex=True) #Convert variation of disebabkan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ditalo)\b", " ditala ", regex=True) #Convert variation of ditala'
tweet_text['text'] = tweet_text['text'].replace(r"\b(doso)\b", " dosa ", regex=True) #Convert variation of dosa'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dukmai)\b", " duk mai ", regex=True) #Convert variation of duk mai'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mah)\b", " mai ", regex=True) #Convert variation of mai'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ekoq)\b", " ekor ", regex=True) #Convert variation of ekor'
tweet_text['text'] = tweet_text['text'].replace(r"\b(incik)\b", " encik ", regex=True) #Convert variation of encik'
tweet_text['text'] = tweet_text['text'].replace(r"\b(poto)\b", " foto ", regex=True) #Convert variation of foto'
tweet_text['text'] = tweet_text['text'].replace(r"\b(terimokasih|tkasih)\b", " terima kasih ", regex=True) #Convert variation of terima kasih'
tweet_text['text'] = tweet_text['text'].replace(r"\b(terigt|tringat)\b", " teringat ", regex=True) #Convert variation of teringat'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tepaksa|tpksa)\b", " terpaksa ", regex=True) #Convert variation of terpaksa'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tamboh)\b", " tambah ", regex=True) #Convert variation of tambah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(nari)\b", " tari ", regex=True) #Convert variation of tari'
tweet_text['text'] = tweet_text['text'].replace(r"\b(gajoh)\b", " gajah ", regex=True) #Convert variation of gajah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(gji)\b", " gaji ", regex=True) #Convert variation of gaji'
tweet_text['text'] = tweet_text['text'].replace(r"\b(gaseng)\b", " gasing ", regex=True) #Convert variation of gasing'
tweet_text['text'] = tweet_text['text'].replace(r"\b(gayuang)\b", " gayung ", regex=True) #Convert variation of gayung'
tweet_text['text'] = tweet_text['text'].replace(r"\b(galanggang)\b", " gelanggang ", regex=True) #Convert variation of gelanggang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(geo|geog)\b", " geografi ", regex=True) #Convert variation of geografi'
tweet_text['text'] = tweet_text['text'].replace(r"\b(gule)\b", " gula ", regex=True) #Convert variation of gula'
tweet_text['text'] = tweet_text['text'].replace(r"\b(gunuang)\b", " gunung ", regex=True) #Convert variation of gunung'
tweet_text['text'] = tweet_text['text'].replace(r"\b(hijo)\b", " hijau ", regex=True) #Convert variation of hijau'
tweet_text['text'] = tweet_text['text'].replace(r"\b(utang)\b", " hutang ", regex=True) #Convert variation of hutang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(jahak)\b", " jahat ", regex=True) #Convert variation of jahat'
tweet_text['text'] = tweet_text['text'].replace(r"\b(jajo)\b", " jaja ", regex=True) #Convert variation of jaja'
tweet_text['text'] = tweet_text['text'].replace(r"\b(jntung)\b", " jantung ", regex=True) #Convert variation of jantung'
tweet_text['text'] = tweet_text['text'].replace(r"\b(jejauh|jaoh)\b", " jauh ", regex=True) #Convert variation of jauh'
tweet_text['text'] = tweet_text['text'].replace(r"\b(jejok)\b", " jejak ", regex=True) #Convert variation of jejak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(jiranku)\b", " jiran aku ", regex=True) #Convert variation of jiran aku'
tweet_text['text'] = tweet_text['text'].replace(r"\b(jumat)\b", " jumaat ", regex=True) #Convert variation of jumaat'
tweet_text['text'] = tweet_text['text'].replace(r"\b(juruh)\b", " lurus ", regex=True) #Convert variation of lurus'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kaco)\b", " kacau ", regex=True) #Convert variation of kacau'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kakloh)\b", " kaklah ", regex=True) #Convert variation of kaklah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kmi|kamik|kmk)\b", " kami ", regex=True) #Convert variation of kami'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kangaq)\b", " kangar ", regex=True) #Convert variation of kangar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kasik)\b", " kasi ", regex=True) #Convert variation of kasi'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kataku)\b", " kata aku ", regex=True) #Convert variation of kata aku'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kawoh)\b", " kawah ", regex=True) #Convert variation of kawah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kayo)\b", " kaya ", regex=True) #Convert variation of kaya'
tweet_text['text'] = tweet_text['text'].replace(r"\b(keatas)\b", " ke atas ", regex=True) #Convert variation of ke atas'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kebenda)\b", " ke benda ", regex=True) #Convert variation of ke benda'
tweet_text['text'] = tweet_text['text'].replace(r"\b(karumah)\b", " ke rumah ", regex=True) #Convert variation of ke rumah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kesudah)\b", " ke sudah ", regex=True) #Convert variation of ke sudah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ketimbang)\b", " ke timbang ", regex=True) #Convert variation of ke timbang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kcek)\b", " kecek ", regex=True) #Convert variation of kecek'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kursi)\b", " kerusi ", regex=True) #Convert variation of kerusi'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kuciwa)\b", " kecewa ", regex=True) #Convert variation of kecewa'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kjayaan)\b", " kejayaan ", regex=True) #Convert variation of kejayaan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(klambi)\b", " kelambi ", regex=True) #Convert variation of kelambi'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kelapo)\b", " kelapa ", regex=True) #Convert variation of kelapa'
tweet_text['text'] = tweet_text['text'].replace(r"\b(marin)\b", " kelmarin ", regex=True) #Convert variation of kelmarin'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kmas)\b", " kemas ", regex=True) #Convert variation of kemas'
tweet_text['text'] = tweet_text['text'].replace(r"\b(km)\b", " kilometer ", regex=True) #Convert variation of kilometer'
tweet_text['text'] = tweet_text['text'].replace(r"\b(klk)\b", " kelik ", regex=True) #Convert variation of kelik'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kmbali)\b", " kembali ", regex=True) #Convert variation of kembali'
tweet_text['text'] = tweet_text['text'].replace(r"\b(hebey)\b", " hebeh ", regex=True) #Convert variation of hebeh'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kenuri)\b", " kenduri ", regex=True) #Convert variation of kenduri'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kpntingn)\b", " kepentingan ", regex=True) #Convert variation of kepentingan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kepong)\b", " kepung ", regex=True) #Convert variation of kepung'
tweet_text['text'] = tweet_text['text'].replace(r"\b(keraih)\b", " keras ", regex=True) #Convert variation of keras'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ketas)\b", " kertas ", regex=True) #Convert variation of kertas'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ktwakan)\b", " ketawakan ", regex=True) #Convert variation of ketawakan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ketok)\b", " ketuk ", regex=True) #Convert variation of ketuk'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kntrol)\b", " kontrol ", regex=True) #Convert variation of kontrol'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kupi)\b", " kopi ", regex=True) #Convert variation of kopi'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kb)\b", " kota bharu ", regex=True) #Convert variation of kota bharu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(klcentral)\b", " kuala lumpur central ", regex=True) #Convert variation of kuala lumpur central'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kuningku)\b", " kuning aku ", regex=True) #Convert variation of kuning aku'
tweet_text['text'] = tweet_text['text'].replace(r"\b(lenge)\b", " lengan ", regex=True) #Convert variation of lengan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(lpak)\b", " lepak ", regex=True) #Convert variation of lepak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(letiaq)\b", " leter ", regex=True) #Convert variation of leter'
tweet_text['text'] = tweet_text['text'].replace(r"\b(luok)\b", " luak ", regex=True) #Convert variation of luak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(lukeh)\b", " lukis ", regex=True) #Convert variation of lukis'
tweet_text['text'] = tweet_text['text'].replace(r"\b(lomba)\b", " lumba ", regex=True) #Convert variation of lumba'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mchang)\b", " machang ", regex=True) #Convert variation of machang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mhkamah)\b", " mahkamah ", regex=True) #Convert variation of mahkamah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mango)\b", " manga ", regex=True) #Convert variation of manga'
tweet_text['text'] = tweet_text['text'].replace(r"\b(marilah)\b", " mari la ", regex=True) #Convert variation of mari la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(memasing)\b", " masing ", regex=True) #Convert variation of masing'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mejo)\b", " meja ", regex=True) #Convert variation of meja'
tweet_text['text'] = tweet_text['text'].replace(r"\b(melake)\b", " melaka ", regex=True) #Convert variation of melaka'
tweet_text['text'] = tweet_text['text'].replace(r"\b(malamar)\b", " melamar ", regex=True) #Convert variation of melamar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(malawan)\b", " melawan ", regex=True) #Convert variation of melawan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mberi)\b", " memberi ", regex=True) #Convert variation of memberi'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mamperbaiki)\b", " memperbaiki ", regex=True) #Convert variation of memperbaiki'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mananti)\b", " menanti ", regex=True) #Convert variation of menanti'
tweet_text['text'] = tweet_text['text'].replace(r"\b(menegoq)\b", " menegur ", regex=True) #Convert variation of menegur'
tweet_text['text'] = tweet_text['text'].replace(r"\b(menakan)\b", " menekan ", regex=True) #Convert variation of menekan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mngada)\b", " mengada ", regex=True) #Convert variation of mengada'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mengajaq)\b", " mengajak ", regex=True) #Convert variation of mengajak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ngaji)\b", " mengaji ", regex=True) #Convert variation of mengaji'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ngaku|ngakuk)\b", " mengaku ", regex=True) #Convert variation of mengaku'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ngalah)\b", " mengalah ", regex=True) #Convert variation of mengalah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mngalahkan)\b", " mengalahkan ", regex=True) #Convert variation of mengalahkan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mangamalkan)\b", " mengamalkan ", regex=True) #Convert variation of mengamalkan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(nganggur)\b", " menganggur ", regex=True) #Convert variation of menganggur'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ngejah)\b", " mengeja ", regex=True) #Convert variation of mengeja'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mngejar)\b", " mengejar ", regex=True) #Convert variation of mengejar'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ngeliat)\b", " mengeliat ", regex=True) #Convert variation of mengeliat'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ngenang)\b", " mengenang ", regex=True) #Convert variation of mengenang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ngepong)\b", " mengepung ", regex=True) #Convert variation of mengepung'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mngganggu)\b", " mengganggu ", regex=True) #Convert variation of mengganggu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ngadap)\b", " menghadap ", regex=True) #Convert variation of menghadap'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ngigau)\b", " mengigau ", regex=True) #Convert variation of mengigau'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ngilai)\b", " mengilai ", regex=True) #Convert variation of mengilai'
tweet_text['text'] = tweet_text['text'].replace(r"\b(zomba)\b", " zumba ", regex=True) #Convert variation of zumba'
tweet_text['text'] = tweet_text['text'].replace(r"\b(wyg)\b", " wayang ", regex=True) #Convert variation of wayang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(warne)\b", " warna ", regex=True) #Convert variation of warna'
tweet_text['text'] = tweet_text['text'].replace(r"\b(wanito)\b", " wanita ", regex=True) #Convert variation of wanita'
tweet_text['text'] = tweet_text['text'].replace(r"\b(usio)\b", " usia ", regex=True) #Convert variation of usia'
tweet_text['text'] = tweet_text['text'].replace(r"\b(usya)\b", " usha ", regex=True) #Convert variation of usha'
tweet_text['text'] = tweet_text['text'].replace(r"\b(uboh)\b", " ubah ", regex=True) #Convert variation of ubah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(nusuk)\b", " tusuk ", regex=True) #Convert variation of tusuk'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tunggui)\b", " tunggul ", regex=True) #Convert variation of tunggul'
tweet_text['text'] = tweet_text['text'].replace(r"\b(numpang)\b", " tumpang ", regex=True) #Convert variation of tumpang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tjuan)\b", " tujuan ", regex=True) #Convert variation of tujuan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tdung)\b", " tudung ", regex=True) #Convert variation of tudung'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tuo)\b", " tua ", regex=True) #Convert variation of tua'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tylu)\b", " tiru ", regex=True) #Convert variation of tiru'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tipulah)\b", " tipu la ", regex=True) #Convert variation of tipu la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(nipu)\b", " tipu ", regex=True) #Convert variation of tipu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(timbng)\b", " timbang ", regex=True) #Convert variation of timbang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tidolh)\b", " tidur la ", regex=True) #Convert variation of tidur la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(truk)\b", " teruk ", regex=True) #Convert variation of teruk'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pebetui)\b", " membetulkan ", regex=True) #Convert variation of membetulkan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tarang)\b", " terang ", regex=True) #Convert variation of terang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tanang)\b", " tenang ", regex=True) #Convert variation of tenang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tmpahan)\b", " tempahan ", regex=True) #Convert variation of tempahan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(timbago)\b", " tembaga ", regex=True) #Convert variation of tembaga'
tweet_text['text'] = tweet_text['text'].replace(r"\b(teluih)\b", " telus ", regex=True) #Convert variation of telus'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tkan)\b", " tekan ", regex=True) #Convert variation of tekan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tawor)\b", " tawar ", regex=True) #Convert variation of tawar
tweet_text['text'] = tweet_text['text'].replace(r"\b(pakso)\b", " paksa ", regex=True) #Convert variation of paksa
tweet_text['text'] = tweet_text['text'].replace(r"\b(tahik)\b", " tahi ", regex=True) #Convert variation of tahi
tweet_text['text'] = tweet_text['text'].replace(r"\b(taboh)\b", " tabur ", regex=True) #Convert variation of tabur
tweet_text['text'] = tweet_text['text'].replace(r"\b(syiok)\b", " syok ", regex=True) #Convert variation of syok
tweet_text['text'] = tweet_text['text'].replace(r"\b(musholla)\b", " surau ", regex=True) #Convert variation of surau
tweet_text['text'] = tweet_text['text'].replace(r"\b(sukahati|sukati)\b", " suka hati ", regex=True) #Convert variation of suka hati
tweet_text['text'] = tweet_text['text'].replace(r"\b(somi|suben)\b", " suami ", regex=True) #Convert variation of suami
tweet_text['text'] = tweet_text['text'].replace(r"\b(suamiku|somiku|subenku)\b", " suami aku ", regex=True) #Convert variation of suami aku
tweet_text['text'] = tweet_text['text'].replace(r"\b(soklan)\b", " soalan ", regex=True) #Convert variation of soalan
tweet_text['text'] = tweet_text['text'].replace(r"\b(selim)\b", " slim ", regex=True) #Convert variation of slim
tweet_text['text'] = tweet_text['text'].replace(r"\b(siotbdosbpzosjxks)\b", " siot ", regex=True) #Convert variation of siot
tweet_text['text'] = tweet_text['text'].replace(r"\b(singgoh)\b", " singgah ", regex=True) #Convert variation of singgah
tweet_text['text'] = tweet_text['text'].replace(r"\b(syial)\b", " sial ", regex=True) #Convert variation of sial
tweet_text['text'] = tweet_text['text'].replace(r"\b(shampo)\b", " shampu ", regex=True) #Convert variation of shampu
tweet_text['text'] = tweet_text['text'].replace(r"\b(sewe|sewo)\b", " sewa ", regex=True) #Convert variation of sewa
tweet_text['text'] = tweet_text['text'].replace(r"\b(sept)\b", " september ", regex=True) #Convert variation of september
tweet_text['text'] = tweet_text['text'].replace(r"\b(sepek)\b", " sepet ", regex=True) #Convert variation of sepet
tweet_text['text'] = tweet_text['text'].replace(r"\b(segal)\b", " sengal ", regex=True) #Convert variation of sengal
tweet_text['text'] = tweet_text['text'].replace(r"\b(snang|sng|sene|seney)\b", " senang ", regex=True) #Convert variation of senang
tweet_text['text'] = tweet_text['text'].replace(r"\b(senaq|snak)\b", " senak ",regex=True) #Convert variation of senak
tweet_text['text'] = tweet_text['text'].replace(r"\b(smentara)\b", " sementara ",regex=True) #Convert variation of sementara
tweet_text['text'] = tweet_text['text'].replace(r"\b(smbunyi)\b", " sembunyi ",regex=True) #Convert variation of sembunyi
tweet_text['text'] = tweet_text['text'].replace(r"\b(smbuhkan|smbuhkn)\b", " sembuhkan ",regex=True) #Convert variation of sembuhkan
tweet_text['text'] = tweet_text['text'].replace(r"\b(sembilang)\b", " sembilan ",regex=True) #Convert variation of sembilan
tweet_text['text'] = tweet_text['text'].replace(r"\b(slimut)\b", " selimut ",regex=True) #Convert variation of selimut
tweet_text['text'] = tweet_text['text'].replace(r"\b(selsema)\b", " selesema ",regex=True) #Convert variation of selesema
tweet_text['text'] = tweet_text['text'].replace(r"\b(slera|slere|selere)\b", " selera ",regex=True) #Convert variation of selera
tweet_text['text'] = tweet_text['text'].replace(r"\b(selate)\b", " selatan ",regex=True) #Convert variation of selatan
tweet_text['text'] = tweet_text['text'].replace(r"\b(slamber)\b", " selamba ",regex=True) #Convert variation of selamba
tweet_text['text'] = tweet_text['text'].replace(r"\b(seghonok)\b", " seronok ",regex=True) #Convert variation of seronok
tweet_text['text'] = tweet_text['text'].replace(r"\b(sekso|sekse)\b", " seksa ",regex=True) #Convert variation of seksa
tweet_text['text'] = tweet_text['text'].replace(r"\b(sbtu)\b", " sabtu ",regex=True) #Convert variation of sabtu
tweet_text['text'] = tweet_text['text'].replace(r"\b(seumoq)\b", " seumur ",regex=True) #Convert variation of seumur
tweet_text['text'] = tweet_text['text'].replace(r"\b(stitik)\b", " setitik ",regex=True) #Convert variation of setitik
tweet_text['text'] = tweet_text['text'].replace(r"\b(stgh)\b", " setengah ",regex=True) #Convert variation of setengah
tweet_text['text'] = tweet_text['text'].replace(r"\b(stahun)\b", " setahun ",regex=True) #Convert variation of setahun
tweet_text['text'] = tweet_text['text'].replace(r"\b(sekapung)\b", " sekampung ",regex=True) #Convert variation of sekampung
tweet_text['text'] = tweet_text['text'].replace(r"\b(sajalan)\b", " sejalan ",regex=True) #Convert variation of sejalan
tweet_text['text'] = tweet_text['text'].replace(r"\b(sgulung)\b", " segulung ",regex=True) #Convert variation of segulung
tweet_text['text'] = tweet_text['text'].replace(r"\b(sdekah)\b", " sedekah ",regex=True) #Convert variation of sedekah
tweet_text['text'] = tweet_text['text'].replace(r"\b(sebuk)\b", " sibuk ",regex=True) #Convert variation of sibuk
tweet_text['text'] = tweet_text['text'].replace(r"\b(qeyak)\b", " riak ",regex=True) #Convert variation of riak
tweet_text['text'] = tweet_text['text'].replace(r"\b(spantas)\b", " sepantas ",regex=True) #Convert variation of sepantas
tweet_text['text'] = tweet_text['text'].replace(r"\b(spatah)\b", " sepatah ",regex=True) #Convert variation of sepatah
tweet_text['text'] = tweet_text['text'].replace(r"\b(nampu)\b", " mampu ",regex=True) #Convert variation of mampu
tweet_text['text'] = tweet_text['text'].replace(r"\b(sedaih)\b", " sedas ",regex=True) #Convert variation of sedas
tweet_text['text'] = tweet_text['text'].replace(r"\b(segitu)\b", " sebegitu ",regex=True) #Convert variation of sebegitu
tweet_text['text'] = tweet_text['text'].replace(r"\b(sbegini)\b", " sebegini ",regex=True) #Convert variation of sebegini
tweet_text['text'] = tweet_text['text'].replace(r"\b(sebare)\b", " sebarang ",regex=True) #Convert variation of sebarang
tweet_text['text'] = tweet_text['text'].replace(r"\b(sebabok)\b", " sebabak ",regex=True) #Convert variation of sebabak
tweet_text['text'] = tweet_text['text'].replace(r"\b(sygkan)\b", " sayangkan ",regex=True) #Convert variation of sayangkan
tweet_text['text'] = tweet_text['text'].replace(r"\b(sawapandey)\b", " sawah padi ",regex=True) #Convert variation of sawah padi
tweet_text['text'] = tweet_text['text'].replace(r"\b(sngka)\b", " sangka ",regex=True) #Convert variation of sangka
tweet_text['text'] = tweet_text['text'].replace(r"\b(smpaikan)\b", " sampaikan ",regex=True) #Convert variation of sampaikan
tweet_text['text'] = tweet_text['text'].replace(r"\b(rupo)\b", " rupa ",regex=True) #Convert variation of rupa
tweet_text['text'] = tweet_text['text'].replace(r"\b(rusing)\b", " runsing ",regex=True) #Convert variation of runsing
tweet_text['text'] = tweet_text['text'].replace(r"\b(rezki|rizki|rejeki)\b", " rezeki ",regex=True) #Convert variation of rezeki
tweet_text['text'] = tweet_text['text'].replace(r"\b(rehatpastu)\b", " rehat lepas itu ",regex=True) #Convert variation of rehat lepas itu
tweet_text['text'] = tweet_text['text'].replace(r"\b(rawe)\b", " rawa ",regex=True) #Convert variation of rawa
tweet_text['text'] = tweet_text['text'].replace(r"\b(rmps)\b", " rampas ",regex=True) #Convert variation of rampas
tweet_text['text'] = tweet_text['text'].replace(r"\b(ramalen)\b", " ramalan ",regex=True) #Convert variation of ramalan
tweet_text['text'] = tweet_text['text'].replace(r"\b(qran)\b", " quran ",regex=True) #Convert variation of quran
tweet_text['text'] = tweet_text['text'].replace(r"\b(putiah)\b", " putih ",regex=True) #Convert variation of putih
tweet_text['text'] = tweet_text['text'].replace(r"\b(pusaro)\b", " pusara ",regex=True) #Convert variation of pusara
tweet_text['text'] = tweet_text['text'].replace(r"\b(muji)\b", " puji ",regex=True) #Convert variation of puji
tweet_text['text'] = tweet_text['text'].replace(r"\b(posa)\b", " puasa ",regex=True) #Convert variation of puasa
tweet_text['text'] = tweet_text['text'].replace(r"\b(proj)\b", " projek ",regex=True) #Convert variation of projek
tweet_text['text'] = tweet_text['text'].replace(r"\b(prof)\b", " profesor ",regex=True) #Convert variation of profesor
tweet_text['text'] = tweet_text['text'].replace(r"\b(poleh)\b", " polis ",regex=True) #Convert variation of polis
tweet_text['text'] = tweet_text['text'].replace(r"\b(pijok)\b", " pijak ",regex=True) #Convert variation of pijak
tweet_text['text'] = tweet_text['text'].replace(r"\b(pihok)\b", " pihak ",regex=True) #Convert variation of pihak
tweet_text['text'] = tweet_text['text'].replace(r"\b(perot)\b", " perut ",regex=True) #Convert variation of perut
tweet_text['text'] = tweet_text['text'].replace(r"\b(persoala)\b", " persoalan ",regex=True) #Convert variation of persoalan
tweet_text['text'] = tweet_text['text'].replace(r"\b(pniagaan)\b", " perniagaan ",regex=True) #Convert variation of perniagaan
tweet_text['text'] = tweet_text['text'].replace(r"\b(perkare)\b", " perkara ",regex=True) #Convert variation of perkara
tweet_text['text'] = tweet_text['text'].replace(r"\b(pribadi)\b", " peribadi ",regex=True) #Convert variation of peribadi
tweet_text['text'] = tweet_text['text'].replace(r"\b(prasan)\b", " perasan ",regex=True) #Convert variation of perasan
tweet_text['text'] = tweet_text['text'].replace(r"\b(pnyakit)\b", " penyakit ",regex=True) #Convert variation of penyakit
tweet_text['text'] = tweet_text['text'].replace(r"\b(penoh)\b", " penuh ",regex=True) #Convert variation of penuh
tweet_text['text'] = tweet_text['text'].replace(r"\b(pensel)\b", " pensil ",regex=True) #Convert variation of pensil
tweet_text['text'] = tweet_text['text'].replace(r"\b(pnjajah)\b", " penjajah ",regex=True) #Convert variation of penjajah
tweet_text['text'] = tweet_text['text'].replace(r"\b(pengotoq|ngotor)\b", " pengotor ",regex=True) #Convert variation of pengotor
tweet_text['text'] = tweet_text['text'].replace(r"\b(pngorbanan)\b", " pengorbanan ",regex=True) #Convert variation of pengorbanan
tweet_text['text'] = tweet_text['text'].replace(r"\b(pnglaman)\b", " pengalaman ",regex=True) #Convert variation of pengalaman
tweet_text['text'] = tweet_text['text'].replace(r"\b(pencapaye)\b", " pencapaian ",regex=True) #Convert variation of pencapaian
tweet_text['text'] = tweet_text['text'].replace(r"\b(penampaq)\b", " penampar ",regex=True) #Convert variation of penampar
tweet_text['text'] = tweet_text['text'].replace(r"\b(plik)\b", " pelik ",regex=True) #Convert variation of pelik
tweet_text['text'] = tweet_text['text'].replace(r"\b(pelaq)\b", " pelak ",regex=True) #Convert variation of pelak
tweet_text['text'] = tweet_text['text'].replace(r"\b(muor|muo)\b", " muar ",regex=True) #Convert variation of muar
tweet_text['text'] = tweet_text['text'].replace(r"\b(basamo)\b", " bersama ",regex=True) #Convert variation of bersama
tweet_text['text'] = tweet_text['text'].replace(r"\b(sebijik)\b", " sebiji ",regex=True) #Convert variation of sebiji
tweet_text['text'] = tweet_text['text'].replace(r"\b(pocah)\b", " pecah ",regex=True) #Convert variation of pecah
tweet_text['text'] = tweet_text['text'].replace(r"\b(psar)\b", " pasar ",regex=True) #Convert variation of pasar
tweet_text['text'] = tweet_text['text'].replace(r"\b(pesengan)\b", " pasangan ",regex=True) #Convert variation of pasangan
tweet_text['text'] = tweet_text['text'].replace(r"\b(parid)\b", " parit ",regex=True) #Convert variation of parit
tweet_text['text'] = tweet_text['text'].replace(r"\b(pareh)\b", " parah ",regex=True) #Convert variation of parah
tweet_text['text'] = tweet_text['text'].replace(r"\b(pepaya)\b", " papaya ",regex=True) #Convert variation of papaya
tweet_text['text'] = tweet_text['text'].replace(r"\b(palapeh)\b", " pelapis ",regex=True) #Convert variation of pelapis
tweet_text['text'] = tweet_text['text'].replace(r"\b(pacik)\b", " pakcik ",regex=True) #Convert variation of pakcik
tweet_text['text'] = tweet_text['text'].replace(r"\b(pokdare)\b", " pak saudara ",regex=True) #Convert variation of pak saudara
tweet_text['text'] = tweet_text['text'].replace(r"\b(pahalo)\b", " pahala ",regex=True) #Convert variation of pahala
tweet_text['text'] = tweet_text['text'].replace(r"\b(padey)\b", " padi ",regex=True) #Convert variation of padi
tweet_text['text'] = tweet_text['text'].replace(r"\b(pdg)\b", " padang ",regex=True) #Convert variation of padang
tweet_text['text'] = tweet_text['text'].replace(r"\b(otaq)\b", " otak ",regex=True) #Convert variation of otak
tweet_text['text'] = tweet_text['text'].replace(r"\b(nynyi)\b", " nyanyi ",regex=True) #Convert variation of nyanyi
tweet_text['text'] = tweet_text['text'].replace(r"\b(ngara)\b", " negara ",regex=True) #Convert variation of negara
tweet_text['text'] = tweet_text['text'].replace(r"\b(sekpalah)\b", " sek apa la ",regex=True) #Convert variation of sek apa la
tweet_text['text'] = tweet_text['text'].replace(r"\b(mncul)\b", " muncul ",regex=True) #Convert variation of muncul
tweet_text['text'] = tweet_text['text'].replace(r"\b(mlut)\b", " mulut ",regex=True) #Convert variation of mulut
tweet_text['text'] = tweet_text['text'].replace(r"\b(mudo)\b", " muda ",regex=True) #Convert variation of muda
tweet_text['text'] = tweet_text['text'].replace(r"\b(mlek)\b", " molek ",regex=True) #Convert variation of molek
tweet_text['text'] = tweet_text['text'].replace(r"\b(mnyak)\b", " minyak ",regex=True) #Convert variation of minyak
tweet_text['text'] = tweet_text['text'].replace(r"\b(mnum)\b", " minum ",regex=True) #Convert variation of minum
tweet_text['text'] = tweet_text['text'].replace(r"\b(mgijinkan)\b", " mengizinkan ",regex=True) #Convert variation of mengizinkan
tweet_text['text'] = tweet_text['text'].replace(r"\b(ngumpat)\b", " mengumpat ",regex=True) #Convert variation of mengumpat
tweet_text['text'] = tweet_text['text'].replace(r"\b(ngundi)\b", " mengundi ",regex=True) #Convert variation of mengundi
tweet_text['text'] = tweet_text['text'].replace(r"\b(manjulang)\b", " menjulang ",regex=True) #Convert variation of menjulang
tweet_text['text'] = tweet_text['text'].replace(r"\b(ngorat|ngurat)\b", " mengurat ",regex=True) #Convert variation of mengurat
tweet_text['text'] = tweet_text['text'].replace(r"\b(ngusik)\b", " mengusik ",regex=True) #Convert variation of mengusik
tweet_text['text'] = tweet_text['text'].replace(r"\b(nyampah)\b", " menyampah ",regex=True) #Convert variation of menyampah
tweet_text['text'] = tweet_text['text'].replace(r"\b(nyelak)\b", " menyelak ",regex=True) #Convert variation of menyelak
tweet_text['text'] = tweet_text['text'].replace(r"\b(menyembuhka)\b", " menyembuhkan ",regex=True) #Convert variation of menyembuhkan
tweet_text['text'] = tweet_text['text'].replace(r"\b(nyempat)\b", " menyempat ",regex=True) #Convert variation of menyempat
tweet_text['text'] = tweet_text['text'].replace(r"\b(nyerang)\b", " menyerang ",regex=True) #Convert variation of menyerang
tweet_text['text'] = tweet_text['text'].replace(r"\b(nyimpan)\b", " menyimpan ",regex=True) #Convert variation of menyimpan
tweet_text['text'] = tweet_text['text'].replace(r"\b(mnyorok)\b", " menyorok ",regex=True) #Convert variation of menyorok
tweet_text['text'] = tweet_text['text'].replace(r"\b(nyusoh)\b", " menyusah ",regex=True) #Convert variation of menyusah
tweet_text['text'] = tweet_text['text'].replace(r"\b(meraong)\b", " meraung ",regex=True) #Convert variation of meraung
tweet_text['text'] = tweet_text['text'].replace(r"\b(mrungut)\b", " merungut ",regex=True) #Convert variation of merungut
tweet_text['text'] = tweet_text['text'].replace(r"\b(nkena)\b", " nak kena ",regex=True) #Convert variation of nak kena
tweet_text['text'] = tweet_text['text'].replace(r"\b(naklena)\b", " nak lena ",regex=True) #Convert variation of nak lena
tweet_text['text'] = tweet_text['text'].replace(r"\b(nakmasak)\b", " nak masak ",regex=True) #Convert variation of nak masak
tweet_text['text'] = tweet_text['text'].replace(r"\b(namati|namatey)\b", " nak mati ",regex=True) #Convert variation of nak mati
tweet_text['text'] = tweet_text['text'].replace(r"\b(namo)\b", " nama ",regex=True) #Convert variation of nak nama
tweet_text['text'] = tweet_text['text'].replace(r"\b(nakpi)\b", " nak pergi ",regex=True) #Convert variation of nak pergi
tweet_text['text'] = tweet_text['text'].replace(r"\b(nakperabeh)\b", " nak menghabiskan ",regex=True) #Convert variation of nak menghabiskan
tweet_text['text'] = tweet_text['text'].replace(r"\b(perabeh+|perhabis)\b", " menghabiskan ",regex=True) #Convert variation of menghabiskan
tweet_text['text'] = tweet_text['text'].replace(r"\b(terbakor)\b", " terbakar ",regex=True) #Convert variation of terbakar
tweet_text['text'] = tweet_text['text'].replace(r"\b(terbawak)\b", " terbawa ",regex=True) #Convert variation of terbawa
tweet_text['text'] = tweet_text['text'].replace(r"\b(tbukak)\b", " terbuka ",regex=True) #Convert variation of terbuka
tweet_text['text'] = tweet_text['text'].replace(r"\b(tacapai)\b", " tercapai ",regex=True) #Convert variation of tercapai
tweet_text['text'] = tweet_text['text'].replace(r"\b(thibur)\b", " terhibur ",regex=True) #Convert variation of terhibur
tweet_text['text'] = tweet_text['text'].replace(r"\b(trimbas)\b", " terimbas ",regex=True) #Convert variation of terimbas
tweet_text['text'] = tweet_text['text'].replace(r"\b(terketor)\b", " terketar ",regex=True) #Convert variation of terketar
tweet_text['text'] = tweet_text['text'].replace(r"\b(telalu)\b", " terlalu ",regex=True) #Convert variation of terlalu
tweet_text['text'] = tweet_text['text'].replace(r"\b(terlanggaq)\b", " terlanggar ",regex=True) #Convert variation of terlanggar
tweet_text['text'] = tweet_text['text'].replace(r"\b(talatak)\b", " tak letak ",regex=True) #Convert variation of tak letak
tweet_text['text'] = tweet_text['text'].replace(r"\b(terlior)\b", " terliur ",regex=True) #Convert variation of terliur
tweet_text['text'] = tweet_text['text'].replace(r"\b(tapaku)\b", " terpaku ",regex=True) #Convert variation of terpaku
tweet_text['text'] = tweet_text['text'].replace(r"\b(tpandang)\b", " terpandang ",regex=True) #Convert variation of terpandang
tweet_text['text'] = tweet_text['text'].replace(r"\b(tetingga)\b", " tertinggal ",regex=True) #Convert variation of tertinggal
tweet_text['text'] = tweet_text['text'].replace(r"\b(bilih)\b", " bilis ",regex=True) #Convert variation of bilis
tweet_text['text'] = tweet_text['text'].replace(r"\b(petei)\b", " petai ",regex=True) #Convert variation of petai
tweet_text['text'] = tweet_text['text'].replace(r"\b(sambei)\b", " sambal ",regex=True) #Convert variation of sambal
tweet_text['text'] = tweet_text['text'].replace(r"\b(bs)\b", " bebas ",regex=True) #Convert variation of bebas
tweet_text['text'] = tweet_text['text'].replace(r"\b(emo)\b", " emosional ",regex=True) #Convert variation of emosional
tweet_text['text'] = tweet_text['text'].replace(r"\b(kbai|bye|bubye|kbye)\b", " ok bye ",regex=True) #Convert variation of ok bye
tweet_text['text'] = tweet_text['text'].replace(r"\b(ez)\b", " easy ",regex=True) #Convert variation of easy
#----------------------------------------------------------------------------------------------------------------------
tweet_text['text'] = tweet_text['text'].replace(r"(mule|mulo)\b", " mula ", regex=True) #Convert variation of 'mula'
tweet_text['text'] = tweet_text['text'].replace(r"\b(takat|stkt|stakat|setakad)\b", " setakat ", regex=True) #Convert variation of 'setakat'
tweet_text['text'] = tweet_text['text'].replace(r"\b(plu|prlu|pelu|butuh)\b", " perlu ", regex=True) #Convert variation of 'perlu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kalai|kalu|kalo|klau|klo|klw|klu|karo|kolo)", " kalau\n ", regex=True) #Convert variation of 'kalau'
tweet_text['text'] = tweet_text['text'].replace(r"\b(lgpon|gpun|agpun|agpon|lgpn|lgpun|lagipun)\b", " lagi pun ", regex=True) #Convert variation of 'lagi pun'
tweet_text['text'] = tweet_text['text'].replace(r"\b(lg|lgi|ag|agi|agik|lagik|gik)\b", " lagi ", regex=True) #Convert variation of 'lagi'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ker|ke|ka)\b", "ke\n", regex=True) #Convert variation of 'ke'
tweet_text['text'] = tweet_text['text'].replace(r"\b(macam|mcam|macm|mcm|mace|cem|chem)\b", " macam ", regex=True) #Convert variation of 'macam'
tweet_text['text'] = tweet_text['text'].replace(r"\b(macmna|cmana|cmna|cmne|cmner|cemana|cane|caner|cana|cena|camna)\b", " macam mana ", regex=True) #Convert variation of 'macam mana'
tweet_text['text'] = tweet_text['text'].replace(r"\b(agamo)\b", " agama ", regex=True) #Convert variation of 'agama'
tweet_text['text'] = tweet_text['text'].replace(r"\b(lagumano|lagumana)\b", " lagu mana ", regex=True) #Convert variation of 'lagu mana'
tweet_text['text'] = tweet_text['text'].replace(r"\b(cmtu|centu|camtu)\b", " macam\ntu ", regex=True) #Convert variation of 'macam tu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(amacam)\b", " apa macam ", regex=True) #Convert variation of 'apa macam'
tweet_text['text'] = tweet_text['text'].replace(r"\b(na|mna|mne|mano|mn|mane|ne|ner|memane|ane)\b", " \nmana ", regex=True) #Convert variation of 'mana'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mnakan)\b", " manakan ", regex=True) #Convert variation of 'manakan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(nt|nti|karang|nty|kareng|nantib)\b", " nanti ", regex=True) #Convert variation of 'nanti'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dpt|dpat|dapet|dapak|dapek)\b", " dapat ", regex=True) #Convert variation of 'dapat'
tweet_text['text'] = tweet_text['text'].replace(r"\b(nk|nok|nak)\b", " nak ", regex=True) #Convert variation of 'nak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(poei|pegi|pegey|prgi|pi|gi|p|g)\b", "\npergi ", regex=True) #Convert variation of 'pergi'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pigak)\b", " pergi juga ", regex=True) #Convert variation of 'pergi juga'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bg|bgi)\b", " bagi ", regex=True) #Convert variation of 'bagi'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mg|mng|mang|meme|memei|memng|memey|emang|memg|memamg)\b", " memang ", regex=True) #Convert variation of 'memang'
tweet_text['text'] = tweet_text['text'].replace(r"\b(memanglah|mgla)\b", " memang la", regex=True) #Convert variation of 'memang la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(smpai|smpi|sampey|sampe)\b", " sampai ", regex=True) #Convert variation of 'sampai'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sne|sna|sano)\b", " sana ", regex=True) #Convert variation of 'sana'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dsna|dsne|dsno|disana)\b", " di sana ", regex=True) #Convert variation of 'di sana'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sni)\b", " sini ", regex=True) #Convert variation of 'sini'
tweet_text['text'] = tweet_text['text'].replace(r"\b(disinilah)\b", " di sini ", regex=True) #Convert variation of 'di sini'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ye|yer|yo|iyo|iya)\b", " ya ", regex=True) #Convert variation of 'ya'
tweet_text['text'] = tweet_text['text'].replace(r"\b(yedok)\b", " ya tak ", regex=True) #Convert variation of 'ya tak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(yeke)\b", " ya ke ", regex=True) #Convert variation of 'ya ke'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mau|mhu|maw|mo)\b", " mahu ", regex=True) #Convert variation of 'mahu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kn|key|an|en)\b", " kan ", regex=True) #Convert variation of 'kan'
tweet_text['text'] = tweet_text['text'].replace(r"(kn|key)\b", "kan ", regex=True) #Convert variation of 'kan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pela|palah|pelah|polah)\b", " apa la ", regex=True) #Convert variation of 'apa la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tape|tapa)\b", " tak apa ", regex=True) #Convert variation of 'tak apa'
tweet_text['text'] = tweet_text['text'].replace(r"\b(jela|jelah)\b", " saja la ", regex=True) #Convert variation of 'saja la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sah|soh|se)\b", " usah ", regex=True) #Convert variation of 'usah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(toksah|toksoh|tokse|tkse|doksoh)\b", " tak usah ", regex=True) #Convert variation of 'tak usah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dokleh|tokleh)\b", " tak boleh ", regex=True) #Convert variation of 'tak boleh'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dokgi|dakgi)\b", " tak lagi ", regex=True) #Convert variation of 'tak lagi'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dokpaka)\b", " tak pakai ", regex=True) #Convert variation of 'tak pakai'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dakpun)\b", " tak pun ", regex=True) #Convert variation of 'tak pun'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tadok|tadek|tadak|tarak)\b", " tak ada ", regex=True) #Convert variation of 'tak ada'
tweet_text['text'] = tweet_text['text'].replace(r"\b(gapapa)\b", " tak ada apa ", regex=True) #Convert variation of 'tak ada apa'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tamau)\b", " tak mahu ", regex=True) #Convert variation of 'tak mahu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tanak)\b", " tak nak ", regex=True) #Convert variation of 'tak nak'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dela|delah|delar)\b", " ada la ", regex=True) #Convert variation of 'ada la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bakpe|bakpa|bakpo|bkpe|bkpo|bkpa|ngapain)\b", " buat apa ", regex=True) #Convert variation of 'buat apa'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ape|po|pe|apo|gpo|gapo|papea|pa|pape|ap|apah)\b", " apa\n ", regex=True) #Convert variation of 'apa'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pahal|apahal)\b", " apa hal ", regex=True) #Convert variation of 'apa hal'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tr)(?!a)(?!e)(?!i)(?!o)(?!u)(?!y)", " ter", regex=True) #Convert variation of 'ter'
tweet_text['text'] = tweet_text['text'].replace(r"\b(br)(?!a)(?!e)(?!i)(?!o)(?!u)", " ber", regex=True) #Convert variation of 'ber'
tweet_text['text'] = tweet_text['text'].replace(r"(?<!a)(?<!e)(?<!i)(?<!o)(?<!u)(?<!b)(?<!n)(?<!k)(?<!g)(?<!r)(tu|tuh|tew)\b", "\ntu ", regex=True) #Convert variation of 'tu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tu|tuh|tew|te)\b", " itu ", regex=True) #Convert variation of 'itu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(hoktu)\b", " hok itu ", regex=True) #Convert variation of 'hok itu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(aritu|haritu|hritu|hrtu|ritu|haritok)\b", " hari itu ", regex=True) #Convert variation of 'hari itu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(masatu)\b", " masa itu ", regex=True) #Convert variation of 'masa itu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pulaktu)\b", " pula itu ", regex=True) #Convert variation of 'pula itu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(lagutu)\b", " lagu itu ", regex=True) #Convert variation of 'lagu itu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pasai|psal|psl|psai|pasl|psi)\b", " pasal ", regex=True) #Convert variation of 'pasal'
tweet_text['text'] = tweet_text['text'].replace(r"\b(rehatpastu)\b", " rehat lepas itu ",regex=True) #Convert variation of rehat lepas itu
tweet_text['text'] = tweet_text['text'].replace(r"\b(pasaipa)\b", " pasal apa ", regex=True) #Convert variation of 'pasal apa'
tweet_text['text'] = tweet_text['text'].replace(r"\b(lepaih|lpas|lps|pah|ph|pas|ps)\b", " lepas ", regex=True) #Convert variation of 'lepas'
tweet_text['text'] = tweet_text['text'].replace(r"\b(lpasni)\b", " lepas ni ", regex=True) #Convert variation of 'lepas ni'
tweet_text['text'] = tweet_text['text'].replace(r"\b(terlps)\b", " terlepas ", regex=True) #Convert variation of 'terlepas'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pa+stu|p+ahtu|pstu|lepaihtu|lpastu|lpstu|phtu|pa+tu|petu)\b", " lepas itu ", regex=True) #Convert variation of 'lepas itu'
tweet_text['text'] = tweet_text['text'].replace(r"(?<!a)(?<!e)(?<!i)(?<!r)(?<!o)(?<!u)(nih|nie|nyh|ney|ni|neh|nia|niek|nio)\b", " ni ", regex=True) #Convert variation of 'ni'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ny)\b", " ni ", regex=True) #Convert variation of 'ni'
tweet_text['text'] = tweet_text['text'].replace(r"(?<!e)(?<!i)(?<!o)(?<!u)(?<!c)(?<!f)(?<!g)(?<!k)(?<!l)(?<!m)(?<!n)(?<!p)(?<!h)(?<!r)(?<!s)(?<!t)(?<!y)(?<!b)(?<!w)(?<!ela)(pon|pn|un|pong|pun|pom)\b", " pun ", regex=True) #Convert variation of 'pun'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tula|tule|tulo|tulah|itulah)\b", " itu la ", regex=True) #Convert variation of 'itu la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tudia)\b", " itu dia ", regex=True) #Convert variation of 'itu dia'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dh|da+h|doh+|da+)\b", " dah ", regex=True) #Convert variation of 'dah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dhko|dohko)\b", " dah ke ", regex=True) #Convert variation of 'dah ke'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dala|dalah)\b", " dah la ", regex=True) #Convert variation of 'dah la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(segitu|agitu|gitu|gtu|bgitu|bgtu|gitchew)\b", " begitu ", regex=True) #Convert variation of 'begitu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(taguna)\b", " tak guna ", regex=True) #Convert variation of 'tak guna'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tatau)\b", " tak tahu ", regex=True) #Convert variation of 'tak tahu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(taula)\b", " tahu la ", regex=True) #Convert variation of 'tahu la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(aq|q|ak|ku|haku)\b", " aku ", regex=True) #Convert variation of 'aku'
tweet_text['text'] = tweet_text['text'].replace(r"\b(akula)\b", " aku la ", regex=True) #Convert variation of 'aku la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(alabaler)\b", " alahai bala ", regex=True) #Convert variation of 'alahai bala la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kalini)\b", " kali ini ", regex=True) #Convert variation of 'kali ini'
tweet_text['text'] = tweet_text['text'].replace(r"\b(nilah)\b", " ini la ", regex=True) #Convert variation of 'ini la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(arini|harini|hrini|hrni|rini)\b", " hari ini ", regex=True) #Convert variation of 'hari ini'
tweet_text['text'] = tweet_text['text'].replace(r"\b(lani|loni|leni|lenim|lening)\b", " sekarang ini ", regex=True) #Convert variation of 'sekarang ini'
tweet_text['text'] = tweet_text['text'].replace(r"\b(cmni|cengini|ceni|cenggini)\b", " macam ini ", regex=True) #Convert variation of 'macam ini'
tweet_text['text'] = tweet_text['text'].replace(r"\b(cam|cm)\b", " macam ", regex=True) #Convert variation of 'macam'
tweet_text['text'] = tweet_text['text'].replace(r"\b(cambabi)\b", " macam babi ", regex=True) #Convert variation of 'macam babi'
tweet_text['text'] = tweet_text['text'].replace(r"\b(apani)\b", " apa ini ", regex=True) #Convert variation of 'apa ini'
tweet_text['text'] = tweet_text['text'].replace(r"\b(apatu)\b", " apa itu ", regex=True) #Convert variation of 'apa itu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(apdiaoi)\b", " apa dia oi ", regex=True) #Convert variation of 'apa dia oi'
tweet_text['text'] = tweet_text['text'].replace(r"\b(padiala)\b", " apa dia la ", regex=True) #Convert variation of 'apa dia la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(gapio)\b", " apa dia", regex=True) #Convert variation of 'apa dia'
tweet_text['text'] = tweet_text['text'].replace(r"\b(apekehei)\b", " apa ke hal ", regex=True) #Convert variation of 'apa ke hal'
tweet_text['text'] = tweet_text['text'].replace(r"\b(apelagi)\b", " apalagi ", regex=True) #Convert variation of 'apalagi'
tweet_text['text'] = tweet_text['text'].replace(r"\b(apesal|apsal)\b", " apa pasal ", regex=True) #Convert variation of 'apa pasal'
tweet_text['text'] = tweet_text['text'].replace(r"\b(phl|pehal|papehal)\b", " apa hal ", regex=True) #Convert variation of 'apa hal'
tweet_text['text'] = tweet_text['text'].replace(r"\b(apalah)\b", " apa la ", regex=True) #Convert variation of 'apa la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pebende|pebenda)\b", " apa benda ", regex=True) #Convert variation of 'apa benda'
tweet_text['text'] = tweet_text['text'].replace(r"\b(apatah)\b", " apa entah ", regex=True) #Convert variation of 'apa entah'
tweet_text['text'] = tweet_text['text'].replace(r"\b(pdhl|pdahal)\b", " pada hal ", regex=True) #Convert variation of 'pada hal'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tatahan)\b", " tak tahan ", regex=True) #Convert variation of 'tak tahan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tatakut)\b", " tak takut ", regex=True) #Convert variation of 'tak takut'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tadapat)\b", " tak dapat ", regex=True) #Convert variation of 'tak dapat'
tweet_text['text'] = tweet_text['text'].replace(r"\b(situlah)\b", " situ la ", regex=True) #Convert variation of 'situ la'
tweet_text['text'] = tweet_text['text'].replace(r"\b(disitu)\b", " di situ ", regex=True) #Convert variation of 'di situ'
tweet_text['text'] = tweet_text['text'].replace(r"\b(disini)\b", " di sini ", regex=True) #Convert variation of 'di sini'
tweet_text['text'] = tweet_text['text'].replace(r"\b(letew)\b", " la itu ", regex=True) #Convert variation of 'la itu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(sebabtu)\b", " sebab itu ", regex=True) #Convert variation of 'sebab itu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(mu)\b", " kamu ", regex=True) #Convert variation of 'kamu'
tweet_text['text'] = tweet_text['text'].replace(r"\b(psti)\b", " pasti ", regex=True) #Convert variation of 'pasti'
tweet_text['text'] = tweet_text['text'].replace(r"\b(hingin)\b", " ingin ", regex=True) #Convert variation of 'ingin'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kemain)\b", " bukan main ", regex=True) #Convert variation of 'bukan main'
tweet_text['text'] = tweet_text['text'].replace(r"\b(bol)\b", " bola ", regex=True) #Convert variation of 'bola'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dasyat)\b", " dahsyat ", regex=True) #Convert variation of 'dahsyat'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kdit)\b", " kredit ", regex=True) #Convert variation of 'kredit'
tweet_text['text'] = tweet_text['text'].replace(r"\b(smbuhkan)\b", " sembuhkan ", regex=True) #Convert variation of 'sembuhkan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(ibadah)\b", " ibadat ", regex=True) #Convert variation of 'ibadat '
tweet_text['text'] = tweet_text['text'].replace(r"\b(heran)\b", " hairan ", regex=True) #Convert variation of 'hairan '
tweet_text['text'] = tweet_text['text'].replace(r"\b(hentam)\b", " hantam ", regex=True) #Convert variation of 'hantam'
tweet_text['text'] = tweet_text['text'].replace(r"\b(hangguk)\b", " angguk ", regex=True) #Convert variation of 'angguk'
tweet_text['text'] = tweet_text['text'].replace(r"\b(gntg)\b", " gantung ", regex=True) #Convert variation of 'gantung'
tweet_text['text'] = tweet_text['text'].replace(r"\b(jelek)\b", " jelik ", regex=True) #Convert variation of 'jelik'
tweet_text['text'] = tweet_text['text'].replace(r"\b(tli)\b", " tali ", regex=True) #Convert variation of 'tali'
tweet_text['text'] = tweet_text['text'].replace(r"\b(dikit)\b", " sedikit ", regex=True) #Convert variation of 'sedikit'
tweet_text['text'] = tweet_text['text'].replace(r"\b(vaks|vak)\b", " vaksin ", regex=True) #Convert variation of 'vaksin'
tweet_text['text'] = tweet_text['text'].replace(r"\b(hubunga)\b", " hubungan ", regex=True) #Convert variation of 'hubungan'
tweet_text['text'] = tweet_text['text'].replace(r"\b(kesiha)\b", " kesihatan ", regex=True) #Convert variation of 'kesihatan'
#----------------------------------------------------------------------------------------------------------------------
tweet_text['text'] = tweet_text['text'].replace(r"\b\w{0,1}\b", " ", regex=True) #Remove single character
tweet_text['text'] = tweet_text['text'].replace(r"\s+", " ", regex=True) #Remove multiple spaces
tweet_text['text'] = tweet_text['text'].replace(r"^\s+", "", regex=True) #Remove beginning spaceds
tweet_text['text'] = tweet_text['text'].replace(r"\s+$", "", regex=True) #Remove ending spaces
# tweet_text['text'].str.split(r"\s+", regex=True) #Split strings
#----------------------------------------------------------------------------------------------------------------------



# Show Result

In [ ]:
tweet_text

,text
0,tadi pagi baru vaksin yang ketiga dan tidak ad...
1,gaji sudah dalam bentuk makanan bergizi akomod...
2,ru tpks disahkan vaksin kanker serviks diwajib...
3,jika masih ada pemberian vaksin yang tidak hal...
5,banyak yang tanya soal vaksin hpv kepada saya ...
...,...
10577,vaksin booster dan prokes ketat strategi ampuh...
10579,orang anti vaksin kaya begitu
10580,beer tak kira semua orang tahu penting vaksin ...
10581,vaksin booster dan prokes ketat strategi ampuh...


In [ ]:
# tokenization
def tokenization(text):
    text = word_tokenize(text)
    return text

tweet_text['text'] = tweet_text['text'].apply(lambda x: tokenization(x.lower()))

tweet_text

,text
0,"[tadi, pagi, baru, vaksin, yang, ketiga, dan, ..."
1,"[gaji, sudah, dalam, bentuk, makanan, bergizi,..."
2,"[ru, tpks, disahkan, vaksin, kanker, serviks, ..."
3,"[jika, masih, ada, pemberian, vaksin, yang, ti..."
5,"[banyak, yang, tanya, soal, vaksin, hpv, kepad..."
...,...
10577,"[vaksin, booster, dan, prokes, ketat, strategi..."
10579,"[orang, anti, vaksin, kaya, begitu]"
10580,"[beer, tak, kira, semua, orang, tahu, penting,..."
10581,"[vaksin, booster, dan, prokes, ketat, strategi..."


In [ ]:
# remove stopwords
stop_words = stopwords.words('indonesian')

# tidak, tidaklah, bukan, bukanlah, bukannya
stop_words.remove("tidak")
stop_words.remove("tidaklah")
stop_words.remove("bukan")
stop_words.remove("bukanlah")
stop_words.remove("bukannya")

tweet_text['text'] = tweet_text['text'].apply(lambda x: [item for item in x if item not in stop_words])

tweet_text

,text
0,"[pagi, vaksin, ketiga, tidak, efek, sore, kera..."
1,"[gaji, bentuk, makanan, bergizi, akomodasi, va..."
2,"[ru, tpks, disahkan, vaksin, kanker, serviks, ..."
3,"[pemberian, vaksin, tidak, halal, putusan, mam..."
5,"[vaksin, hpv, yasudah, tulis, singkat, biar, b..."
...,...
10577,"[vaksin, booster, prokes, ketat, strategi, amp..."
10579,"[orang, anti, vaksin, kaya]"
10580,"[beer, orang, vaksin, hpv, kaget, orang, berop..."
10581,"[vaksin, booster, prokes, ketat, strategi, amp..."


Remove English Word

In [ ]:
eng_words = set(nltk.corpus.words.words())

tweet_text['text'] = tweet_text['text'].apply(lambda x: [item for item in x if item not in eng_words])

Remove list from Text

In [ ]:
def remove_token(text):
    text  = ' '.join([str(elem) for elem in text])
    return text

tweet_text['text'] = tweet_text['text'].apply(lambda x : remove_token(x))

tweet_text

,text
0,pagi vaksin ketiga tidak efek kerasa nyeri nyeri
1,gaji bentuk makanan bergizi akomodasi vaksin
2,ru tpks disahkan vaksin kanker serviks diwajib...
3,pemberian vaksin tidak putusan mama menuntut m...
5,vaksin hpv yasudah tulis singkat biar baca
...,...
10577,vaksin prokes ketat ampuh cegah berdiri jokowi
10579,vaksin kaya
10580,vaksin hpv kaget beropini liat
10581,vaksin prokes ketat ampuh cegah berdiri jokowi


In [ ]:
# dropping ALL duplicate values
tweet_text.drop_duplicates(subset ="text", inplace = True)

In [ ]:
tweet_text

,text
0,pagi vaksin ketiga tidak efek kerasa nyeri nyeri
1,gaji bentuk makanan bergizi akomodasi vaksin
2,ru tpks disahkan vaksin kanker serviks diwajib...
3,pemberian vaksin tidak putusan mama menuntut m...
5,vaksin hpv yasudah tulis singkat biar baca
...,...
10564,vaksin tidak membatalkan puasa ramadhan
10565,didu sibuk ngejar vaksin
10570,percepatan vaksinasi bpbd cikarang ajak masyar...
10576,polsek kutsel ayo vaksin ragu


In [ ]:
tweet_text = tweet_text[~tweet_text.text.str.contains("hpv")]

In [ ]:
tweet_text = tweet_text.reset_index()

In [ ]:
del tweet_text['index']

In [ ]:
tweet_text

,text
0,pagi vaksin ketiga tidak efek kerasa nyeri nyeri
1,gaji bentuk makanan bergizi akomodasi vaksin
2,ru tpks disahkan vaksin kanker serviks diwajib...
3,pemberian vaksin tidak putusan mama menuntut m...
4,vaksin sinovac astra pfizer ganggu bangat sih ...
...,...
3116,vaksin tidak membatalkan puasa ramadhan
3117,didu sibuk ngejar vaksin
3118,percepatan vaksinasi bpbd cikarang ajak masyar...
3119,polsek kutsel ayo vaksin ragu


In [ ]:
tweet_text.to_csv('21_April_2022_cleaned_vaksin_data.csv')

In [ ]:
# tweet_text.to_csv('/content/drive/MyDrive/FYP/Data/NEW_cleaned_vaksin_data.csv')